In [82]:
from FlagEmbedding import BGEM3FlagModel, FlagReranker
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.schema import Document
from langchain.text_splitter import MarkdownTextSplitter
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import torch
import os
import tempfile
import pymupdf4llm
import arxiv
import requests
import re
import shutil
from typing import Optional, Tuple, List

from tqdm import tqdm
import pandas as pd
import time
import numpy as np

from rouge_score import rouge_scorer, scoring
from sacrebleu import corpus_bleu, BLEU

In [16]:
references = ["Это эталонная суммаризация."]
hypotheses = ["Это сгенерированная суммаризация."]
bleu = sacrebleu.corpus_bleu(hypotheses, [references], tokenize='intl')
print("BLEU:", bleu.score)

BLEU: 35.35533905932737


# Оценка Retriever

In [2]:
def extract_arxiv_id(url: str) -> str:
    """
    Извлекает arXiv ID из URL.

    Args:
        url (str): Ссылка на статью arXiv.

    Returns:
        str: Извлечённый ID или None, если не удалось.
    """
    match = re.search(r'arxiv\.org/(abs|pdf)/([0-9]+\.[0-9]+)(v\d+)?', url)
    return match.group(2) if match else None


def download_pdf(arxiv_id: str, save_path: str = "article.pdf") -> str:
    """
    Скачивает PDF-файл по arXiv ID и сохраняет на диск.

    Args:
        arxiv_id (str): Идентификатор статьи на arXiv.
        save_path (str): Путь, по которому сохранить файл.

    Returns:
        Tuple[str, str]: Путь до сохранённого файла и название статьи.
    """
    client = arxiv.Client()
    search = arxiv.Search(id_list=[arxiv_id])
    result = next(client.results(search))
    pdf_url = result.pdf_url

    response = requests.get(pdf_url)
    with open(save_path, "wb") as f:
        f.write(response.content)
    return save_path, result.title
    

def trim_markdown_after_section(md_text: str, section="references", aliases: Optional[List[str]] = None) -> str:
    """
    Удаляет всё из markdown-текста начиная с указанной секции (по умолчанию — References).

    Args:
        md_text (str): Исходный Markdown.
        section (str): Основной заголовок для поиска.
        aliases (List[str]): Альтернативные варианты заголовка.

    Returns:
        str: Markdown до найденного заголовка.
    """
    all_keys = [section.lower()] + [a.lower() for a in aliases or []]

    pattern = re.compile(
        rf'^#+\s*\**({"|".join(map(re.escape, all_keys))})\**.*$',
        re.IGNORECASE | re.MULTILINE
    )
    match = pattern.search(md_text)
    return md_text[:match.start()] if match else md_text


def clean_markdown_for_rag(md_raw: str) -> str:
    """
    Очищает markdown от визуального мусора, лишних символов и пустых строк.

    Args:
        md_text (str): Исходный Markdown текст.

    Returns:
        str: Очищенный и нормализованный Markdown.
    """
    lines = md_raw.splitlines()
    cleaned_lines = []

    for line in lines:
        stripped = line.strip()

        if stripped == "-----":
            continue

        if re.fullmatch(r'[\d\s\.,%]+', stripped):
            continue

        line = re.sub(r'\*\[(,|\*|\d+)\]\*', '', line)
        line = re.sub(r'\*(,+|\s*[,\.])\*', '', line)
        line = re.sub(r'^(#{1,6})\s*\*\*(.*?)\*\*\s*$', r'\1 \2', line)
        line = re.sub(r'\*{3,}', '**', line)
        line = re.sub(r'\s{2,}', ' ', line)

        cleaned_lines.append(line)

    # Склеиваем обратно и нормализуем множественные пустые строки
    cleaned_text = "\n".join(cleaned_lines)

    # Заменяем 3+ пустых строк подряд на 2
    cleaned_text = re.sub(r'\n{3,}', '\n\n', cleaned_text)

    return cleaned_text

def extract_section(md_text: str, section: str, aliases: Optional[List[str]] = None) -> str:
    """
    Извлекает текст секции по заголовку, учитывая альтернативные варианты.

    Args:
        md_text (str): Markdown-документ.
        section (str): Название секции.
        aliases (List[str]): Возможные альтернативы (например, 'summary' для 'abstract').

    Returns:
        str: Извлечённый текст секции.
    """
    lines = md_text.splitlines()
    start_idx, end_idx = None, None

    all_keys = [section.lower()] + [a.lower() for a in aliases or []]

    # 1. Найдём заголовок секции
    for i, line in enumerate(lines):
        line_clean = re.sub(r'[\*\#]', '', line).strip().lower()
        for key in all_keys:
            if re.match(rf'^(\d+[\.\d+]*)?\s*{re.escape(key)}$', line_clean):
                start_idx = i + 1
                break
        if start_idx:
            break

    if start_idx is None:
        return ""

    # 2. Поиск конца секции — следующего заголовка
    for j in range(start_idx, len(lines)):
        next_line_clean = re.sub(r'[\*\#]', '', lines[j]).strip().lower()
        if re.match(r'^(\d+[\.\d+]*)?\s+\w+', next_line_clean):
            end_idx = j
            break

    section_lines = lines[start_idx:end_idx] if end_idx else lines[start_idx:]
    return "\n".join(section_lines).strip()



def split_into_documents(md_cleaned: str, arxiv_id: str, title: str):
    splitter = MarkdownTextSplitter(chunk_size=800, chunk_overlap=100)
    docs = splitter.create_documents([md_cleaned])
    for doc in docs:
        doc.metadata = {"arxiv_id": arxiv_id, "title": title}
    return docs



def get_vector_store(docs, persist_directory="test_chroma_db"):
    model_name = "BAAI/bge-m3"
    embedding_model = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"})
    vectordb = Chroma.from_documents(documents=docs, embedding=embedding_model, persist_directory=persist_directory)
    return vectordb



def retrieve_and_rerank(question, vectordb, arxiv_id, top_k=5, top_n=2):
    """
    Извлекает top_k кандидатов из Chroma и реранжирует их с помощью FlagReranker.
    Возвращает top_n наиболее релевантных фрагментов.
    
    :param question: Вопрос пользователя
    :param vectordb: Объект Chroma
    :param top_k: Сколько фрагментов извлекать из Chroma
    :param top_n: Сколько лучших возвращать после rerank
    :return: Список кортежей (Document, score)
    """
    retriever = vectordb.as_retriever(search_kwargs={"k": top_k, "filter": {"arxiv_id": arxiv_id}})
    candidates = retriever.get_relevant_documents(question)

    reranker = FlagReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)
    pairs = [[question, doc.page_content] for doc in candidates]
    scores = reranker.compute_score(pairs, normalize=True)

    reranked = sorted(zip(candidates, scores), key=lambda x: -x[1])
    
    # print("🔍 Rerank результаты:")
    # for i, (doc, score) in enumerate(reranked):
    #     print(f"#Чанк{i+1} — {doc.page_content}...")

    return reranked[:top_n]

### Статья Attention is all you need

In [9]:
# скачиваем и сохраняем статью attention is all you need
arxiv_url = "https://arxiv.org/abs/1706.03762"
arxiv_id = extract_arxiv_id(arxiv_url)

pdf_path = f'./articles/{arxiv_id}.pdf'
pdf_path, title = download_pdf(arxiv_id, pdf_path)
md_raw = pymupdf4llm.to_markdown(pdf_path)
md_trimmed = trim_markdown_after_section(md_raw)
md_cleaned = clean_markdown_for_rag(md_trimmed)
abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])

In [12]:
# разбиваем на чанки
docs = split_into_documents(md_cleaned, arxiv_id, title)

In [13]:
# добавляем в бд
vectordb = get_vector_store(docs)

C:\Users\kiril\AppData\Local\Temp\ipykernel_12204\2220767186.py:168: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"})


In [27]:
questions = ['Какой оптимайзер и гиперпараметры использовались в модели?',
            'Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?',
             'Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?',
             'Как устроена структура encoder и decoder в Transformer?',
             'Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?',
             'Какие функции выполняют позиционные кодировки и почему были выбраны синусоиды?',
             'В чем преимущества self-attention по сравнению с рекуррентными и сверточными слоями в плане вычислительной сложности и длины пути зависимостей?',
             'Какие техники регуляризации использовались при обучении Transformer?',
             'Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?',
             'Почему Transformer позволяет более эффективную параллелизацию при обучении?'
            ]

In [59]:
n=9
user_question = questions[n]
top_chunks = retrieve_and_rerank(user_question, vectordb, arxiv_id, top_k=10, top_n=2)
context = f'Вопрос: {user_question}' + "\n\n" + "\n\n".join(f'Чанк {i+1}: {doc.page_content}' for i, (doc, _) in enumerate(top_chunks))

print(context)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Вопрос: Почему Transformer позволяет более эффективную параллелизацию при обучении?

Чанк 1: Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [ 2, 19 ]. In all but a few cases [ 27 ], however, such attention mechanisms
are used in conjunction with a recurrent network.

In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
### 2 Background

Чанк 2: [ 16 ], ByteNet [ 18 ] and ConvS2S [ 9 ], all of which use convolutional neural networks as basic building
block, computing hidden representation

In [70]:
vector_marks_1 = [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]

In [81]:
import numpy as np
print(f"Оценка ретривера на 10 вопросах: {np.mean(vector_marks_1) * 100}%")

Оценка ретривера на 10 вопросах: 90.0%


### Статья TTRL

In [60]:
# скачиваем и сохраняем статью 
arxiv_url = "https://arxiv.org/pdf/2504.16084"
arxiv_id = extract_arxiv_id(arxiv_url)

pdf_path = f'./articles/{arxiv_id}.pdf'
pdf_path, title = download_pdf(arxiv_id, pdf_path)
md_raw = pymupdf4llm.to_markdown(pdf_path)
md_trimmed = trim_markdown_after_section(md_raw)
md_cleaned = clean_markdown_for_rag(md_trimmed)
abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])

In [63]:
# разбиваем на чанки
docs = split_into_documents(md_cleaned, arxiv_id, title)

In [64]:
# добавляем в бд
vectordb = get_vector_store(docs)

In [65]:
questions = ['В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?',
             'Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?',
             'Какую роль играет метод большинства голосов (majority voting) в архитектуре TTRL?',
             'Какие улучшения производительности были достигнуты с помощью TTRL на бенчмарке AIME 2024?',
             'Как TTRL сравнивается с RL на размеченных тестовых данных по эффективности?',
             'Почему TTRL способен работать даже при неточных оценках меток?',
             'Какие модели и бенчмарки использовались для оценки эффективности TTRL?',
             'Какие факторы могут привести к сбою или неэффективности TTRL?',
             'Как TTRL масштабируется при увеличении размера модели?',
             'С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?'
            ]

In [76]:
n=9
user_question = questions[n]
top_chunks = retrieve_and_rerank(user_question, vectordb, arxiv_id, top_k=10, top_n=2)
context = f'Вопрос: {user_question}' + "\n\n" + "\n\n".join(f'Чанк {i+1}: {doc.page_content}' for i, (doc, _) in enumerate(top_chunks))

print(context)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Вопрос: С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?

Чанк 1: **TTRL is compatible with different RL algorithms.** Figure 4 presents the results. We
apply **TTRL** using PPO (Schulman et al., 2017) on MATH-500 to assess its compatibility
with different reinforcement learning algorithms. The performance trajectories of PPO and
GRPO are closely aligned. Compared to GRPO, PPO yields more stable outcomes while
achieving similar overall performance.

**3.3** **Training Dynamics**

Чанк 2: In this paper, we propose Test-Time Reinforcement Learning ( **TTRL** ), a novel framework
for training large language models with Reinforcement Learning (RL) on test data without
access to ground-truth labels. A key component of **TTRL** is its majority voting reward
function, which generates rule-based rewards based on consensus among model predictions.
Our experiments demonstrate the strong potential of **TTRL**, achieving consistent improve

In [69]:
vector_marks_2 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [83]:
import numpy as np
print(f"Оценка ретривера на 10 вопросах: {np.mean(vector_marks_2) * 100}%")

Оценка ретривера на 10 вопросах: 100.0%


### Статья Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

In [43]:
# скачиваем и сохраняем статью 
arxiv_url = "https://arxiv.org/pdf/2005.11401"
arxiv_id = extract_arxiv_id(arxiv_url)

pdf_path = f'./articles/{arxiv_id}.pdf'
pdf_path, title = download_pdf(arxiv_id, pdf_path)
md_raw = pymupdf4llm.to_markdown(pdf_path)
md_trimmed = trim_markdown_after_section(md_raw)
md_cleaned = clean_markdown_for_rag(md_trimmed)
abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])

In [44]:
# разбиваем на чанки
docs = split_into_documents(md_cleaned, arxiv_id, title)

In [45]:
# добавляем в бд
vectordb = get_vector_store(docs)

C:\Users\kiril\AppData\Local\Temp\ipykernel_8780\737500078.py:150: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"})


In [46]:
questions = ['Какая модель используется в качестве генератора в RAG?',
             'Какая модель используется в качестве ретривера в RAG и как она инициализируется?',
             'Чем отличаются архитектуры RAG-Token и RAG-Sequence?',
             'Какой объем документов используется в векторном индексе Wikipedia?',
             'Какие задачи были использованы для оценки RAG-моделей?',
             'Почему авторы считают генерацию предпочтительной по сравнению с извлечением при ответе на вопросы?',
             'Какую производительность показал RAG на задаче Jeopardy Question Generation по сравнению с BART?',
             'Как осуществляется совместное обучение генератора и ретривера в RAG?',
             'Какой эффект даёт "hot-swapping" индекса документов в RAG?',
             'Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?'
            ]

In [66]:
# тут повторно записал статью поэтому выкидываю дубликаты по чанкам

n=9
user_question = questions[n]
top_chunks = retrieve_and_rerank(user_question, vectordb, arxiv_id, top_k=10, top_n=4)
del top_chunks[1:3]
context = f'Вопрос: {user_question}' + "\n\n" + "\n\n".join(f'Чанк {i+1}: {doc.page_content}' for i, (doc, _) in enumerate(top_chunks))

print(context)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Вопрос: Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?

Чанк 1: **2.5** **Decoding**

At test time, RAG-Sequence and RAG-Token require different ways to approximate arg max *y* *p* ( *y|x* ) .

**RAG-Token** The RAG-Token model can be seen as a standard, autoregressive seq2seq generator with transition probability: *p* *[′]* *θ* [(] *[y]* *[i]* *[|][x, y]* [1:] *[i][−]* [1] [) =][ �] *z∈* top- *k* ( *p* ( *·|x* )) *[p]* *[η]* [(] *[z]* *[i]* *[|][x]* [)] *[p]* *[θ]* [(] *[y]* *[i]* *[|][x, z]* *[i]* *[, y]* [1:] *[i][−]* [1] [)] [ To]

decode, we can plug *p* *[′]* *θ* [(] *[y]* *[i]* *[|][x, y]* [1:] *[i][−]* [1] [)][ into a standard beam decoder.]

Чанк 2: **2.1** **Models**

**RAG-Sequence Model** The RAG-Sequence model uses the same retrieved document to generate
the complete *sequence* . Technically, it treats the retrieved document as a single latent variable that
is marginalized to get the seq2seq probability *p* ( *y|x* ) via a top-K approximatio

In [67]:
vector_marks_3 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [68]:
import numpy as np
print(f"Оценка ретривера на 10 вопросах: {np.mean(vector_marks_3) * 100}%")

Оценка ретривера на 10 вопросах: 100.0%


### Статья Efficient Estimation of Word Representations in Vector Space

In [72]:
# скачиваем и сохраняем статью 
arxiv_url = "https://arxiv.org/pdf/1301.3781"
arxiv_id = extract_arxiv_id(arxiv_url)

In [82]:
questions = ['Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?',
             'В чём основное различие между архитектурами CBOW и Skip-gram?',
             'Какие типы отношений включены в Semantic-Syntactic Word Relationship test set?',
             'Какие значения точности показала модель Skip-gram на семантических и синтаксических подзадачах?',
             'Какой подход используется в Skip-gram модели для выбора контекстных слов?',
             'Какова формула вычислительной сложности модели CBOW?',
             'Почему авторы отказались от использования скрытого слоя в новых архитектурах?',
             'Как масштабировалась тренировка моделей в распределённой системе DistBelief?',
             'Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?',
             'Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?'
            ]

In [92]:
# тут повторно записал статью поэтому выкидываю дубликаты по чанкам

n=9
user_question = questions[n]
top_chunks = retrieve_and_rerank(user_question, vectordb, arxiv_id, top_k=10, top_n=2)
context = f'Вопрос: {user_question}' + "\n\n" + "\n\n".join(f'Чанк {i+1}: {doc.page_content}' for i, (doc, _) in enumerate(top_chunks))

print(context)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Вопрос: Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?

Чанк 1: Somewhat surprisingly, it was found that similarity of word representations goes beyond simple
syntactic regularities. Using a word offset technique where simple algebraic operations are performed on the word vectors, it was shown for example that *vector(”King”) - vector(”Man”) + vec-*
*tor(”Woman”)* results in a vector that is closest to the vector representation of the word *Queen* [20].

Чанк 2: We evaluate the overall accuracy for all question types, and for each question type separately (semantic, syntactic). Question is assumed to be correctly answered only if the closest word to the
vector computed using the above method is exactly the same as the correct word in the question;
synonyms are thus counted as mistakes. This also means that reaching 100% accuracy is likely
to be impossible, as the current models do not have any input information about word morphology.
However, we believe

In [101]:
vector_marks_4 = [0, 1, 0, 1, 1, 1, 1, 1, 1, 1]

In [102]:
import numpy as np
print(f"Оценка ретривера на 10 вопросах: {np.mean(vector_marks_4) * 100}%")

Оценка ретривера на 10 вопросах: 80.0%


### Статья LADDER: SELF-IMPROVING LLMS THROUGH RECURSIVE PROBLEM DECOMPOSITION

In [111]:
# скачиваем и сохраняем статью 
arxiv_url = "https://arxiv.org/pdf/2503.00735"
arxiv_id = extract_arxiv_id(arxiv_url)

In [112]:
questions = ['В чём заключается ключевая идея фреймворка LADDER?',
             'Какой метод используется для верификации правильности решений интегралов в LADDER?',
             'Как осуществляется генерация вариантов задач в LADDER?',
             'Что такое TTRL и чем он отличается от LADDER?',
             'Какие результаты достигнуты моделью Qwen2.5 7B на экзамене MIT Integration Bee после применения LADDER и TTRL?',
             'Какая функция вознаграждения используется в GRPO в рамках обучения в LADDER?',
             'Почему RL без использования вариантов задач показывает плохие результаты на задачах интегрирования?',
             'Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?',
             'Как LADDER масштабирует обучение без увеличения размера модели?',
             'Какие ограничения или сложности были замечены при генерации вариантов задач?'
            ]

In [122]:
n=9
user_question = questions[n]
top_chunks = retrieve_and_rerank(user_question, vectordb, arxiv_id, top_k=10, top_n=2)
context = f'Вопрос: {user_question}' + "\n\n" + "\n\n".join(f'Чанк {i+1}: {doc.page_content}' for i, (doc, _) in enumerate(top_chunks))

print(context)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Вопрос: Какие ограничения или сложности были замечены при генерации вариантов задач?

Чанк 1: Analysis of the remaining unsolved problems reveals nuanced challenges in the model’s mathematical integration
capabilities. The model demonstrates difficulty with integrals requiring multiple technique combinations and novel
substitution patterns. These complex scenarios highlight limitations in the current approach, suggesting that while the
variant generation successfully teaches individual techniques, synthesizing these methods for more intricate problems
remains a challenge.

Чанк 2: *x* [2] + 1
Original problem:
� *x* [4] + 2 *x* [2] + 1 *[dx]*

*x* [2]
Level 1 variant:
� *x* [4] + 1 *[dx]*

Level 2 variant:
� *x* [2] + 1 *[dx]*

Level 3 variant:
� *x* [2] *[ dx]*

Quality control presented a significant challenge in our variant generation process. While we prompted the model to
generate "easier" or "equivalent" variants, the actual difficulty of the resulting integrals often deviated su

In [123]:
vector_marks_5 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [124]:
import numpy as np
print(f"Оценка ретривера на 10 вопросах: {np.mean(vector_marks_5) * 100}%")

Оценка ретривера на 10 вопросах: 100.0%


## Оценка ретривера на 5 статьях (50 вопросов)

In [126]:
print(f"Оценка ретривера: {np.mean(vector_marks_1 + vector_marks_2 + vector_marks_3 + vector_marks_4 + vector_marks_5) * 100}%")

Оценка ретривера: 94.0%


# Оценка суммаризации

## Сбор и загрузка данных в ChromaDB

In [3]:
urls = ['https://arxiv.org/pdf/2504.16084',
        'https://arxiv.org/abs/1706.03762',
        'https://arxiv.org/pdf/2112.02242',
        'https://arxiv.org/pdf/2405.12819',
        'https://arxiv.org/pdf/2005.11401',
        'https://arxiv.org/pdf/1301.3781',
        'https://arxiv.org/pdf/2501.07391',
        'https://arxiv.org/pdf/2503.00735',
        'https://arxiv.org/pdf/2501.19399',
        'https://arxiv.org/pdf/2205.14135']

data_lst = []

In [4]:
for arxiv_url in tqdm(urls):
    arxiv_id = extract_arxiv_id(arxiv_url)
    
    pdf_path = f'./articles/{arxiv_id}.pdf'
    pdf_path, title = download_pdf(arxiv_id, pdf_path)
    md_raw = pymupdf4llm.to_markdown(pdf_path)
    md_trimmed = trim_markdown_after_section(md_raw)
    md_cleaned = clean_markdown_for_rag(md_trimmed)
    abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
    conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])
    
    final = abstract + "\n\n" + conclusion
    data_lst.append(final)

    docs = split_into_documents(md_cleaned, arxiv_id, title)
    vectordb = get_vector_store(docs)

C:\Users\kiril\AppData\Local\Temp\ipykernel_6364\737500078.py:150: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"})
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:44<00:00, 10.44s/it]


In [13]:
# Эталонные суммаризации
gt = ['В данной работе представлен метод Test-Time Reinforcement Learning (TTRL) — новый подход к обучению больших языковых моделей (LLMs) с использованием обучения с подкреплением на неразмеченных тестовых данных. Ключевая идея TTRL заключается в применении majority voting для оценки вознаграждения без доступа к истинным меткам. Метод позволяет LLM самостоятельно улучшать свои ответы, опираясь на собственные предсказания. Эксперименты показывают, что TTRL обеспечивает значительный прирост производительности, включая увеличение pass@1 на 159% на AIME 2024, и показывает сопоставимые результаты с моделями, обученными на размеченных данных. Таким образом, TTRL открывает путь к масштабируемому RL с самогенерируемыми наградами и применению в более широком спектре задач.',
      'В статье представлен Transformer — первая архитектура для преобразования последовательностей, основанная исключительно на механизме внимания, без рекуррентных и сверточных слоёв. Transformer достигает нового уровня качества в машинном переводе на задачах WMT 2014 (28.4 BLEU для EN→DE и 41.8 BLEU для EN→FR), при этом обучается быстрее и параллелится лучше, чем предыдущие модели. Авторы подчеркивают потенциал архитектуры для широкого применения, включая неречевые модальности, и планируют дальнейшие исследования в области локального внимания и более эффективной генерации',
      'В работе предлагается подход к обучению рекомендательных систем с учётом долгосрочной памяти пользователей на основе неявного фидбека. Авторы представляют онлайн-алгоритм, обновляющий модель по пользовательским блокам взаимодействий. Основной вклад заключается в стратегии фильтрации пользователей по стационарности и устойчивости поведения, что позволяет существенно повысить качество рекомендаций (MAP, NDCG) при обучении на сильно сокращенном, но более информативном подмножестве данных. Метод показал значительные улучшения, особенно на больших датасетах',
      'Авторы представляют первый систематический обзор применения больших языковых моделей (LLMs) в задачах NLP, предлагая единую таксономию подходов: с замороженными параметрами (parameter-frozen) и с адаптацией (parameter-tuning). Работа также выделяет новые исследовательские направления и вызовы, с целью стимулировать дальнейшие прорывы в области. В дополнение, создан публичный ресурс для отслеживания последних публикаций, что делает обзор не только аналитическим, но и практическим инструментом для исследователей',
      'Авторы представляют общий подход к дообучению RAG-моделей, сочетающих параметрическую память (предобученные seq2seq модели) и непараметрическую (векторный индекс Википедии, доступный через нейронный ретривер). Они сравнивают две версии: с фиксированными и токен-зависимыми извлечениями. Модель демонстрирует новое состояние искусства в задачах открытого вопросно-ответного поиска и генерирует более точный и разнообразный текст, чем чисто параметрические seq2seq модели, показывая потенциал RAG в задачах, требующих доступа к знаниям',
      'Авторы представляют два простых и эффективных архитектурных подхода — CBOW и Skip-gram — для обучения векторных представлений слов на масштабных корпусах. Эти модели обеспечивают значительное улучшение качества синтаксических и семантических представлений при низких вычислительных затратах, позволяя обучать векторы даже на триллионе слов. Векторы достигают state-of-the-art результатов в задачах оценки семантической близости и находят применение в ряде NLP-задач, включая машинный перевод, анализ тональности и расширение знаний в базах данных. Работа демонстрирует потенциал простых моделей как базового элемента для будущих приложений в NLP',
      'В работе представлено систематическое исследование архитектур и компонентов Retrieval-Augmented Generation (RAG) систем. Авторы изучают влияние таких факторов, как размер модели, дизайн промпта, размер чанков, объём базы знаний, стратегии извлечения, query expansion и Contrastive In-Context Learning, в том числе в мультиязычном контексте. Эксперименты показывают, как конфигурации RAG влияют на качество ответов, предлагая практические рекомендации по созданию эффективных и адаптируемых RAG-систем для различных задач. Код и реализации опубликованы в открытом доступе.',
      'Авторы представляют LADDER — фреймворк, улучшающий способности языковых моделей к решению задач с помощью рекурсивной декомпозиции, обучения с подкреплением с верифицируемыми наградами и его расширения на инференс — TTRL. Метод позволяет моделям самостоятельно упрощать сложные задачи и обучаться на этих вариантах, достигая прорывных результатов в математических задачах (например, рост точности с 1% до 82% на задачах интегрирования и 90% на MIT Integration Bee). Работа подчёркивает ценность стратегического самообучения, проверяемых наград и тестового самоулучшения, демонстрируя потенциал применения подхода в других доменах, таких как синтез программ и автоматическое доказательство теорем.',
      'Scalable-Softmax (SSMax) — это новая замена функции Softmax в слоях внимания Transformer-моделей, разработанная для решения проблемы "затухания внимания" при увеличении размера входного контекста. SSMax сохраняет способность модели фокусироваться на ключевой информации даже при длинных входах, улучшая обобщающую способность по длине, скорость сходимости на этапе предобучения и точность извлечения информации. SSMax можно внедрять как с начала обучения, так и на уже предобученные модели, получая стабильные улучшения. Работа показывает, что SSMax — перспективный кандидат для замены Softmax в будущих архитектурах трансформеров, особенно в задачах с длинным контекстом',
      'Авторы представляют FlashAttention — IO-ориентированный алгоритм точного внимания, оптимизированный для GPU-архитектур. Он использует тайлинг, чтобы минимизировать количество операций чтения и записи между высокоскоростной памятью (HBM) и локальной SRAM, что значительно ускоряет обучение трансформеров. FlashAttention обеспечивает реальный прирост скорости (до 3× на GPT-2, 15% на BERT-large) без ущерба для качества модели. Расширенная версия с блок-спарсностью опережает существующие приближённые методы по скорости и позволяет моделям работать с контекстами до 64K токенов, впервые демонстрируя выше случайного уровня точности на задачах Path-X и Path-256. Метод открывает путь к более длинным, качественным и эффективным трансформерам']

## T-Lite-8B

In [3]:
model_name = "t-bank-ai/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quant_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)
terminators = list({
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>") or tokenizer.eos_token_id,
    })

embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )

reranker = FlagReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)

vectorstore = Chroma(persist_directory="test_chroma_db", embedding_function=embedding_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
C:\Users\kiril\AppData\Local\Temp\ipykernel_27976\295889804.py:12: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
C:\Users\kiril\AppData\Local\Temp\ipykernel_27976\295889804.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="test_chroma_db", embedding_fu

In [ ]:
SYSTEM_MSG = (
    "Ты — помощник по научным статьям. "
    "Сформулируй краткое и точное резюме по тексту ниже."
)

def build_messages(abstract, conclusion) -> list[dict]:
    full_text = f"Abstract:\n{abstract}\n\nConclusion:\n{conclusion}"

    return [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": full_text},
        {"role": "assistant", "content": "Краткое резюме:"},
    ]

In [7]:
outputs = []
times = []
for arxiv_url in tqdm(urls):
    arxiv_id = extract_arxiv_id(arxiv_url)
    
    pdf_path = f'./articles/{arxiv_id}.pdf'
    pdf_path, title = download_pdf(arxiv_id, pdf_path)
    md_raw = pymupdf4llm.to_markdown(pdf_path)
    md_trimmed = trim_markdown_after_section(md_raw)
    md_cleaned = clean_markdown_for_rag(md_trimmed)
    abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
    conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])
    
    messages = build_messages(abstract, conclusion)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, eos_token_id=terminators)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Название статьи: {title}\n')
    print(f'Суммаризация: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:19<02:54, 19.35s/it]

Название статьи: TTRL: Test-Time Reinforcement Learning

Суммаризация:  В статье представлен метод Test-Time Reinforcement Learning ( **TTRL** ), который использует Reinforcement Learning для улучшения больших языковых моделей (LLMs) на основе данных без меток. Основная задача — создание подходящих вознаграждений для обучения на основе консенсуса предсказаний модели. **TTRL** применяет метод большинства голосов для оценки вознаграждений, что показывает значительные улучшения в различных задачах и моделях. В частности, на тесте AIME 2024 **TTRL** увеличивает точность на 159% по сравнению с исходной моделью. Результаты подтверждают эффективность **TTRL** и открывают перспективы для дальнейшего развития самообучающихся систем на основе опыта.




 20%|████████████████▌                                                                  | 2/10 [00:41<02:49, 21.17s/it]

Название статьи: Attention Is All You Need

Суммаризация:  В статье представлен новый архитектурный подход к моделированию последовательностей — Transformer, который полностью основан на механизмах внимания, отказываясь от рекуррентных и свёрточных сетей. В ходе экспериментов на задачах машинного перевода на данных WMT 2014, Transformer показывает значительное улучшение в качестве перевода и скорости обучения по сравнению с традиционными моделями. На англо-немецком направлении модель достигает 28.4 BLEU, превосходя предыдущие результаты, а на англо-французском — 41.8 BLEU, что является новым рекордом для одиночной модели. Авторы также демонстрируют успешное применение Transformer для парсинга текста и подчеркивают перспективы дальнейшего использования и адаптации этой технологии для других задач и данных. 

Ссылки на код и библиотеку: `https://github.com/tensorflow/tensor2tensor` 

Авторы благодарят коллег за помощь и поддержку в разработке и тестировании.




 30%|████████████████████████▉                                                          | 3/10 [00:53<01:59, 17.05s/it]

...

Название статьи: LADDER: Self-Improving LLMs Through Recursive Problem Decomposition

Суммаризация:  В статье представлен LADDER — метод, улучшающий способность моделей решать задачи через рекурсивное разложение проблем и обучение с подкреплением на основе верифицируемых вознаграждений. Включение тестового обучения (TTRL) позволило значительно повысить эффективность в решении математических задач, таких как интеграция, достигнув 90% точности на MIT Integration Bee. Работа подчеркивает важность стратегического обучения и использования проверяемых обратных связей для повышения производительности ИИ. Принципы могут быть применены в различных областях, где есть возможность верификации, и способствуют развитию систем, которые могут самостоятельно улучшать свои навыки в сложных задачах.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:41<00:18, 18.44s/it]

Название статьи: Scalable-Softmax Is Superior for Attention

Суммаризация:  В статье представлен Scalable-Softmax (SSMax) — альтернативный подход к использованию функции Softmax в трансформерных моделях для улучшения работы с длинными контекстами и повышения эффективности внимания. SSMax решает проблему "затухания" внимания, возникающего при увеличении размера входных данных, и улучшает способность модели удерживать внимание на ключевых элементах информации. 

### Основные выводы:
1. **Проблема Softmax**: Максимальный элемент Softmax уменьшается с ростом размера входного вектора, что ухудшает распределение внимания.
2. **SSMax решение**: Предложенная функция SSMax сохраняет внимание на ключевых элементах, даже при больших размерах контекста.
3. **Экспериментальные результаты**:
   - **Ускорение обучения**: Модели с SSMax быстрее снижают потери во время обучения.
   - **Улучшение в длинных контекстах**: SSMax значительно улучшает производительность в длинных последовательностях.
   - **

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:06<00:00, 18.65s/it]

Название статьи: FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness

Суммаризация:  
FlashAttention — это новый подход к реализации внимания в трансформерах, который решает проблему высокой вычислительной и памяти-затратности на длинных последовательностях. В отличие от традиционных методов, FlashAttention учитывает операции ввода-вывода (IO) между уровнями памяти GPU, что позволяет значительно уменьшить количество обращений к высокоскоростной памяти (HBM) и улучшить производительность. 

Основные достижения:
- FlashAttention обеспечивает до 15% ускорения в тренировке BERT-large и до 3-кратного ускорения для GPT-2 по сравнению с текущими рекордами.
- Внедрение в блок-спарсивание внимания позволяет достичь еще большего ускорения и улучшает качество моделей.
- Увеличение контекста в моделях приводит к улучшению метрик (перплексии и точности) и новым возможностям, таким как достижение лучших результатов на задачах с длинными последовательностями (Path-X и Path-256

In [41]:
times

[14.314668655395508,
 17.953752756118774,
 9.457916498184204,
 13.708330154418945,
 13.110188961029053,
 12.668460130691528,
 15.162318706512451,
 12.614119529724121,
 19.203843116760254,
 18.3269145488739]

In [49]:
print(f'Среднее время суммаризации модели Тбанк: {np.mean(times):.0f} сек')

Среднее время суммаризации модели Тбанк: 15 сек


In [15]:
df_summarization = pd.DataFrame()

df_summarization['gt'] = gt
df_summarization['Tbank'] = outputs

In [18]:
df_summarization.to_csv('summarize_examples.csv', index=False)

In [19]:
df_summarization

,gt,Tbank
0,В данной работе представлен метод Test-Time Re...,В статье представлен метод Test-Time Reinforc...
1,В статье представлен Transformer — первая архи...,В статье представлен новый архитектурный подх...
2,В работе предлагается подход к обучению рекоме...,В данной работе исследуется влияние долгосроч...
3,Авторы представляют первый систематический обз...,\n\nИсследование посвящено анализу использова...
4,Авторы представляют общий подход к дообучению ...,"В статье рассматриваются модели, сочетающие п..."
5,Авторы представляют два простых и эффективных ...,В статье представлены два новых архитектурных...
6,В работе представлено систематическое исследов...,\n\nВ статье рассматриваются и анализируются ...
7,"Авторы представляют LADDER — фреймворк, улучша...","В статье представлен LADDER — метод, улучшающ..."
8,Scalable-Softmax (SSMax) — это новая замена фу...,В статье представлен Scalable-Softmax (SSMax)...
9,Авторы представляют FlashAttention — IO-ориент...,\nFlashAttention — это новый подход к реализа...


## YandexGPT5-lite 8B

In [4]:
model_name = "yandex/YandexGPT-5-Lite-8B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quant_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)

embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )

reranker = FlagReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)

vectorstore = Chroma(persist_directory="test_chroma_db", embedding_function=embedding_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
C:\Users\kiril\AppData\Local\Temp\ipykernel_4456\1072813799.py:8: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
C:\Users\kiril\AppData\Local\Temp\ipykernel_4456\1072813799.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="test_chroma_db", embedding_fun

In [8]:
SYSTEM_MSG = (
    "Ты — помощник по научным статьям. "
    "Сформулируй краткое и точное резюме по тексту ниже. Отвечай строго на русском языке"
)

def build_messages(abstract, conclusion) -> list[dict]:
    full_text = f"Abstract:\n{abstract}\n\nConclusion:\n{conclusion}"

    return [
        {"role": "user", "content": SYSTEM_MSG},
        {"role": "user", "content": full_text}
    ]

In [9]:
outputs = []
times = []
for arxiv_url in tqdm(urls):
    arxiv_id = extract_arxiv_id(arxiv_url)
    
    pdf_path = f'./articles/{arxiv_id}.pdf'
    pdf_path, title = download_pdf(arxiv_id, pdf_path)
    md_raw = pymupdf4llm.to_markdown(pdf_path)
    md_trimmed = trim_markdown_after_section(md_raw)
    md_cleaned = clean_markdown_for_rag(md_trimmed)
    abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
    conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])
    
    messages = build_messages(abstract, conclusion)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False)[0]['generated_text']
    
    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Название статьи: {title}\n')
    print(f'Суммаризация: {answer}\n\n')
    outputs.append(answer)

C:\Users\kiril\miniconda3\envs\nlp_finetune\lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\kiril\miniconda3\envs\nlp_finetune\lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 10%|████████▎                                                                          | 1/10 [00:20<03:06, 20.71s/it]

Название статьи: TTRL: Test-Time Reinforcement Learning

Суммаризация:  В статье представлен метод Test-Time Reinforcement Learning (TTRL) для обучения больших языковых моделей (LLM) с помощью RL на немаркированных данных. TTRL использует предварительные знания из предварительно обученных моделей и позволяет им «самоэволюционировать».

Эксперименты показали, что TTRL последовательно улучшает производительность различных моделей и задач. Например, TTRL повысил показатель pass@1 модели Qwen-2.5-Math-7B примерно на 159% на AIME 2024, используя только немаркированные тестовые данные.

Метод TTRL демонстрирует свою эффективность и потенциал для более широкого применения в различных задачах и областях.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 20%|████████████████▌                                                                  | 2/10 [00:44<02:58, 22.29s/it]

Название статьи: Attention Is All You Need

Суммаризация:  В работе представлена новая архитектура нейронной сети — Transformer, основанная исключительно на механизмах внимания, без использования рекуррентных и свёрточных слоёв. Эксперименты по машинному переводу показали, что модели на основе Transformer превосходят по качеству существующие модели, при этом они более параллелизуемы и требуют значительно меньше времени на обучение.

На задачах перевода Transformer демонстрирует лучшие результаты по сравнению с другими архитектурами, обучаясь значительно быстрее. Модель Transformer достигла новых рекордных показателей BLEU на задачах WMT 2014 по переводу с английского на немецкий (28,4 BLEU) и с английского на французский (41,8 BLEU).

Авторы планируют применять модели на основе внимания к другим задачам, а также исследовать локальные механизмы внимания для эффективной обработки больших входных и выходных данных, таких как изображения, аудио и видео.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 30%|████████████████████████▉                                                          | 3/10 [00:54<01:56, 16.68s/it]

...

Название статьи: LADDER: Self-Improving LLMs Through Recursive Problem Decomposition

Суммаризация:  В статье представлен фреймворк LADDER для улучшения способностей языковых моделей к решению задач через рекурсивную декомпозицию задач и обучение с подкреплением с проверяемыми вознаграждениями. Также описано его расширение для времени тестирования (TTRL).

LADDER позволяет моделям генерировать и обучаться на постепенно упрощающихся вариантах сложных задач. Подход демонстрирует значительные улучшения без необходимости архитектурного масштабирования или человеческого надзора.

Эффективность подхода подтверждена на примере математических задач: точность решения интегралов повысилась с 1% до 82%, а на MIT Integration Bee — до 90%.

Принципы, продемонстрированные в статье, могут быть применены в любых областях с чёткими механизмами проверки, от синтеза программ до доказательства теорем.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:27<00:16, 16.30s/it]

Название статьи: Scalable-Softmax Is Superior for Attention

Суммаризация:  В статье предлагается Scalable-Softmax (SSMax) как альтернатива функции Softmax в механизмах внимания Transformer. SSMax решает проблему «затухания» внимания и улучшает обобщение для длинных контекстов.

Эксперименты показали, что модели с SSMax достигают более быстрого снижения потерь при предварительном обучении и значительно улучшают производительность в длинных контекстах и при извлечении ключевой информации. Модели, использующие SSMax с начала предварительного обучения, демонстрируют наибольшую способность к обобщению, но и уже обученные модели могут получить преимущества от замены Softmax на SSMax во время или после предварительного обучения.

SSMax является перспективным подходом для решения ограничений Softmax в механизмах внимания Transformer, особенно для задач с расширенными контекстами.




You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:48<00:00, 16.88s/it]

Название статьи: FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness

Суммаризация:  В статье предлагается FlashAttention — алгоритм точного внимания, учитывающий операции ввода-вывода (IO-aware). Он использует тайлинг для сокращения количества операций чтения/записи между памятью GPU и сокращает время работы трансформеров по сравнению с существующими методами.

FlashAttention обеспечивает ускорение работы трансформеров по сравнению с базовыми алгоритмами: на 15% для BERT-large, в 3 раза для GPT-2 и в 2,4 раза для long-range arena. Также алгоритм позволяет увеличить длину контекста в трансформерах, что повышает качество моделей и открывает новые возможности.




In [10]:
times

[14.131155252456665,
 16.678635597229004,
 7.474043846130371,
 9.699246644973755,
 10.073513269424438,
 12.498071432113647,
 8.384318590164185,
 14.539063930511475,
 13.785437822341919,
 11.562705516815186]

In [11]:
print(f'Среднее время суммаризации модели YandexGPT5: {np.mean(times):.0f} сек')

Среднее время суммаризации модели YandexGPT5: 12 сек


In [10]:
df_summarization = pd.read_csv('summarize_examples.csv')

In [14]:
df_summarization['YandexGPT'] = outputs

In [16]:
df_summarization.to_csv('summarize_examples.csv', index=False)

In [23]:
df_summarization

,gt,Tbank,YandexGPT
0,В данной работе представлен метод Test-Time Re...,В статье представлен метод Test-Time Reinforc...,В статье представлен метод Test-Time Reinforce...
1,В статье представлен Transformer — первая архи...,В статье представлен новый архитектурный подх...,В работе представлена новая архитектура нейрон...
2,В работе предлагается подход к обучению рекоме...,В данной работе исследуется влияние долгосроч...,В статье изучается влияние долговременной памя...
3,Авторы представляют первый систематический обз...,\n\nИсследование посвящено анализу использова...,В статье представлен системный обзор больших я...
4,Авторы представляют общий подход к дообучению ...,"В статье рассматриваются модели, сочетающие п...",В статье рассматривается проблема ограниченног...
5,Авторы представляют два простых и эффективных ...,В статье представлены два новых архитектурных...,В статье предложены две новые архитектуры моде...
6,В работе представлено систематическое исследов...,\n\nВ статье рассматриваются и анализируются ...,В статье представлены усовершенствованные диза...
7,"Авторы представляют LADDER — фреймворк, улучша...","В статье представлен LADDER — метод, улучшающ...",В статье представлен фреймворк LADDER для улуч...
8,Scalable-Softmax (SSMax) — это новая замена фу...,В статье представлен Scalable-Softmax (SSMax)...,В статье предлагается Scalable-Softmax (SSMax)...
9,Авторы представляют FlashAttention — IO-ориент...,\nFlashAttention — это новый подход к реализа...,В статье предлагается FlashAttention — алгорит...


## QWEN 3

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
quant_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #quantization_config=quant_config,
    torch_dtype="auto",
    device_map="cuda"
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda


In [6]:
SYSTEM_MSG = (
    "Ты — помощник по научным статьям. "
    "Сформулируй краткое и точное резюме по тексту ниже. Отвечай строго на русском языке"
)

def build_messages(abstract, conclusion) -> list[dict]:
    full_text = f"Abstract:\n{abstract}\n\nConclusion:\n{conclusion}"

    return [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": full_text},
        {"role": "assistant", "content": "Краткое резюме:"},
    ]

In [9]:
outputs = []
times = []
for arxiv_url in tqdm(urls):
    arxiv_id = extract_arxiv_id(arxiv_url)
    
    pdf_path = f'../articles/{arxiv_id}.pdf'
    pdf_path, title = download_pdf(arxiv_id, pdf_path)
    md_raw = pymupdf4llm.to_markdown(pdf_path)
    md_trimmed = trim_markdown_after_section(md_raw)
    md_cleaned = clean_markdown_for_rag(md_trimmed)
    abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
    conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])
    
    messages = build_messages(abstract, conclusion)
    messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, temperature=0.7, top_p=0.8, top_k=20, min_p=0)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Название статьи: {title}\n')
    print(f'Суммаризация: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:22<03:25, 22.83s/it]

Название статьи: TTRL: Test-Time Reinforcement Learning

Суммаризация: В данной работе предложен новый метод обучения больших языковых моделей (LLM) с использованием усиленного обучения (RL) на неотмеченных тестовых данных, названный Test-Time Reinforcement Learning (TTRL). Основная идея заключается в использовании метода голосования большинства для оценки вознаграждений без доступа к истинным меткам. Эксперименты показали, что TTRL значительно улучшает производительность моделей на различных задачах, включая математические задачи, и приближает их к уровню моделей, обученных на данных с истинными метками. TTRL демонстрирует высокую эффективность и потенциал для применения в различных задачах и областях.




 20%|████████████████▌                                                                  | 2/10 [00:41<02:43, 20.40s/it]

Название статьи: Attention Is All You Need

Суммаризация: В данной работе предложен архитектура Transformer, основанная исключительно на механизмах внимания, заменяющая традиционные рекуррентные и свёрточные сети. Эксперименты на задачах машинного перевода показали, что модель достигает более высокого качества перевода, обладает лучшей параллелизуемостью и требует меньше времени на обучение. Transformer установил новые рекорды по BLEU на задачах английско-немецкого и английско-французского перевода. Модель успешно применялась и на других задачах, таких как разбор предложения. Код доступен по ссылке.




 30%|████████████████████████▉                                                          | 3/10 [01:06<02:36, 22.35s/it]

...

Название статьи: LADDER: Self-Improving LLMs Through Recursive Problem Decomposition

Суммаризация: LADDER — это фреймворк, улучшающий способности языковых моделей решать задачи за счёт рекурсивного разложения проблем и обучения с подкреплением с проверяемыми наградами. Его расширение TTRL позволяет модели эффективно улучшать свои навыки в режиме работы. Эксперименты показали значительное повышение способностей к математическому мышлению, что демонстрирует эффективность подхода. Работа основана на трёх ключевых идеях: рекурсивном разложении задач, важности проверяемых наград и стратегическом использовании вычислительных ресурсов. Принципы LADDER могут быть применены в различных областях с возможностью проверки результатов, позволяя моделям систематически расширять свои возможности.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [03:17<00:22, 22.61s/it]

Название статьи: Scalable-Softmax Is Superior for Attention

Суммаризация: В данной работе предложен новый подход — Scalable-Softmax (SSMax), который заменяет классическую функцию Softmax в слоях внимания трансформеров. SSMax решает проблему ухудшения распределения внимания при увеличении длины контекста, позволяя моделям эффективно фокусироваться на ключевой информации. Эксперименты показали, что модели с SSMax демонстрируют улучшенную скорость обучения, лучшую генерализацию на длинные контексты и более высокую точность в задачах извлечения ключевой информации. SSMax может быть легко интегрирован в существующие архитектуры трансформеров и приносит пользу как при обучении с нуля, так и при модификации уже обученных моделей.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:42<00:00, 22.22s/it]

Название статьи: FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness

Суммаризация: FlashAttention — это новый алгоритм точного внимания, учитывающий ограничения памяти (IO-оптимизированный), который значительно ускоряет обработку длинных последовательностей в трансформерах. Он использует тайлинг для минимизации операций чтения/записи между высокопроизводительной памятью HBM и SRAM. FlashAttention позволяет ускорить обучение моделей, таких как BERT-large, GPT-2 и Long Range Arena, и улучшает качество моделей, позволяя обрабатывать более длинные контексты. Расширение FlashAttention — блок-спарсное внимание — обеспечивает еще более высокую производительность при приближенном внимании.




In [6]:
outputs = []
times = []
for arxiv_url in tqdm(urls):
    arxiv_id = extract_arxiv_id(arxiv_url)
    
    pdf_path = f'./articles/{arxiv_id}.pdf'
    pdf_path, title = download_pdf(arxiv_id, pdf_path)
    md_raw = pymupdf4llm.to_markdown(pdf_path)
    md_trimmed = trim_markdown_after_section(md_raw)
    md_cleaned = clean_markdown_for_rag(md_trimmed)
    abstract = extract_section(md_cleaned, "abstract", ["absctract", "summary"])
    conclusion = extract_section(md_cleaned, "conclusion", ["conclusions", "closing remarks"])
    
    messages = build_messages(abstract, conclusion)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, temperature=None, top_p=None, top_k=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Название статьи: {title}\n')
    print(f'Суммаризация: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:24<03:39, 24.43s/it]

Название статьи: TTRL: Test-Time Reinforcement Learning

Суммаризация:   
В статье предложен метод Test-Time Reinforcement Learning (TTRL), позволяющий обучать большие языковые модели с использованием усилительного обучения на неотмеченных тестовых данных. Основная идея заключается в использовании функции награды на основе большинства голосов, основанной на согласии предсказаний модели. Эксперименты показали, что TTRL значительно улучшает производительность на различных задачах, включая математические задачи, и приближает результаты к уровням моделей, обученных на данных с правильными метками.




 20%|████████████████▌                                                                  | 2/10 [00:47<03:08, 23.60s/it]

Название статьи: Attention Is All You Need

Суммаризация:   
Статья представляет архитектуру Transformer — модель, основанную полностью на механизмах внимания, которая заменяет традиционные рекуррентные и свёрточные сети. Эксперименты показали, что Transformer превосходит существующие модели по качеству перевода, обладает лучшей параллелизуемостью и требует меньше времени на обучение. На задачах перевода с английского на немецкий и французкий модель достигла новых рекордов по метрике BLEU. Также Transformer успешно применялся для других задач, таких как синтаксический анализ.




 30%|████████████████████████▉                                                          | 3/10 [01:10<02:43, 23.42s/it]

...

Название статьи: LADDER: Self-Improving LLMs Through Recursive Problem Decomposition

Суммаризация:   
LADDER — это фреймворк, улучшающий способности языковых моделей к решению задач через рекурсивное разложение проблем и обучение с подтвержденными наградами. Его расширение TTRL позволяет модели эффективно улучшать свои навыки в режиме тестирования. Эксперименты показали значительное повышение способностей к математическому мышлению, что демонстрирует эффективность подхода. Работа подчеркивает важность стратегического взаимодействия моделей с окружением, а не только архитектурных инноваций. Принципы LADDER могут быть применены в различных областях с четкими механизмами проверки, способствуя систематическому расширению возможностей ИИ.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [03:27<00:24, 24.34s/it]

Название статьи: Scalable-Softmax Is Superior for Attention

Суммаризация:   
В статье предложен новый метод Scalable-Softmax (SSMax), который улучшает работу внимания в трансформерных моделях, особенно при работе с длинными контекстами. SSMax решает проблему уплощения распределения внимания, характерную для стандартного Softmax, и позволяет моделям лучше фокусироваться на ключевой информации. Эксперименты показали, что модели с SSMax демонстрируют более быструю сходимость, улучшенную обобщаемость на длинные контексты и лучшую точность в задачах извлечения ключевой информации. SSMax может быть легко интегрирован в существующие архитектуры и эффективно работает как при обучении с нуля, так и при замене Softmax в уже обученных моделях.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:47<00:00, 22.77s/it]

Название статьи: FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness

Суммаризация:   
FlashAttention — это алгоритм точного внимания, учитывающий ограничения памяти (IO-оптимизированный), который значительно ускоряет обучение трансформеров, сокращая количество обращений к высокопроизводительной памяти GPU. Он позволяет использовать более длинные контексты, улучшая качество моделей и открывая новые возможности, такие как высокая точность на задачах с очень длинными последовательностями.




In [7]:
times

[18.104891061782837,
 19.647082090377808,
 16.988260507583618,
 12.867859601974487,
 16.346726655960083,
 23.98879098892212,
 13.167678594589233,
 15.985154390335083,
 26.50315833091736,
 11.247574806213379]

In [8]:
print(f'Среднее время суммаризации модели Qwen3: {np.mean(times):.0f} сек')

Среднее время суммаризации модели Qwen3: 17 сек


In [11]:
df_summarization = pd.read_csv('summarize_examples.csv')

In [12]:
outputs = [x.split('\n')[1] for x in outputs]
df_summarization['Qwen3'] = outputs

In [13]:
df_summarization = df_summarization[['gt', 'Tbank', 'YandexGPT', 'Qwen3', 'summarize_best_llm', 'sim_scores_tbank', 'sim_scores_yandex']]

In [14]:
df_summarization

,gt,Tbank,YandexGPT,Qwen3,summarize_best_llm,sim_scores_tbank,sim_scores_yandex
0,В данной работе представлен метод Test-Time Re...,В статье представлен метод Test-Time Reinforc...,В статье представлен метод Test-Time Reinforce...,В данной работе предложен новый метод обучения...,1,0.909943,0.930276
1,В статье представлен Transformer — первая архи...,В статье представлен новый архитектурный подх...,В работе представлена новая архитектура нейрон...,Статья представляет архитектуру Transformer — ...,1,0.797422,0.852751
2,В работе предлагается подход к обучению рекоме...,В данной работе исследуется влияние долгосроч...,В статье изучается влияние долговременной памя...,В статье исследуется влияние долгосрочной памя...,0,0.774107,0.773998
3,Авторы представляют первый систематический обз...,\n\nИсследование посвящено анализу использова...,В статье представлен системный обзор больших я...,Статья представляет собой первый систематическ...,1,0.755156,0.731499
4,Авторы представляют общий подход к дообучению ...,"В статье рассматриваются модели, сочетающие п...",В статье рассматривается проблема ограниченног...,Статья представляет обобщённый подход к тонкой...,0,0.737129,0.688586
5,Авторы представляют два простых и эффективных ...,В статье представлены два новых архитектурных...,В статье предложены две новые архитектуры моде...,В статье предложены два новых архитектурных по...,0,0.739331,0.708769
6,В работе представлено систематическое исследов...,\n\nВ статье рассматриваются и анализируются ...,В статье представлены усовершенствованные диза...,В статье представлены продвинутые дизайн-систе...,0,0.856174,0.903531
7,"Авторы представляют LADDER — фреймворк, улучша...","В статье представлен LADDER — метод, улучшающ...",В статье представлен фреймворк LADDER для улуч...,"LADDER — это фреймворк, улучшающий способности...",1,0.804446,0.914699
8,Scalable-Softmax (SSMax) — это новая замена фу...,В статье представлен Scalable-Softmax (SSMax)...,В статье предлагается Scalable-Softmax (SSMax)...,В статье предложен новый метод Scalable-Softma...,1,0.864461,0.898752
9,Авторы представляют FlashAttention — IO-ориент...,\nFlashAttention — это новый подход к реализа...,В статье предлагается FlashAttention — алгорит...,"FlashAttention — это точный алгоритм внимания,...",0,0.899040,0.914540


In [37]:
n = 9
str_gt = df_summarization['gt'].iloc[n]
str_tbank = df_summarization['Tbank'].iloc[n]
str_yandex = df_summarization['YandexGPT'].iloc[n]
str_qwen = df_summarization['Qwen3'].iloc[n]

print(f'Эталонная суммаризация: {str_gt}\n')
print(f'Суммаризация Tbank: {str_tbank}\n')
print(f'Суммаризация YandexGPT: {str_yandex}\n')  
print(f'Суммаризация Qwen: {str_qwen}')

Эталонная суммаризация: Авторы представляют FlashAttention — IO-ориентированный алгоритм точного внимания, оптимизированный для GPU-архитектур. Он использует тайлинг, чтобы минимизировать количество операций чтения и записи между высокоскоростной памятью (HBM) и локальной SRAM, что значительно ускоряет обучение трансформеров. FlashAttention обеспечивает реальный прирост скорости (до 3× на GPT-2, 15% на BERT-large) без ущерба для качества модели. Расширенная версия с блок-спарсностью опережает существующие приближённые методы по скорости и позволяет моделям работать с контекстами до 64K токенов, впервые демонстрируя выше случайного уровня точности на задачах Path-X и Path-256. Метод открывает путь к более длинным, качественным и эффективным трансформерам

Суммаризация Tbank:  
FlashAttention — это новый подход к реализации внимания в трансформерах, который решает проблему высокой вычислительной и памяти-затратности на длинных последовательностях. В отличие от традиционных методов, Flash

In [6]:
#test
df = pd.read_csv('summarize_examples.csv')

In [12]:
n = 5
str_gt = df['gt'].iloc[n]
str_tbank = df['Tbank'].iloc[n]
str_yandex = df['YandexGPT'].iloc[n]
str_qwen = df['Qwen3'].iloc[n]

print(f'Эталонная суммаризация: {str_gt}\n')
print(f'Суммаризация Tbank: {str_tbank}\n')
print(f'Суммаризация YandexGPT: {str_yandex}\n')  
print(f'Суммаризация Qwen: {str_qwen}')

Эталонная суммаризация: Авторы представляют два простых и эффективных архитектурных подхода — CBOW и Skip-gram — для обучения векторных представлений слов на масштабных корпусах. Эти модели обеспечивают значительное улучшение качества синтаксических и семантических представлений при низких вычислительных затратах, позволяя обучать векторы даже на триллионе слов. Векторы достигают state-of-the-art результатов в задачах оценки семантической близости и находят применение в ряде NLP-задач, включая машинный перевод, анализ тональности и расширение знаний в базах данных. Работа демонстрирует потенциал простых моделей как базового элемента для будущих приложений в NLP

Суммаризация Tbank:  В статье представлены два новых архитектурных решения для создания непрерывных векторных представлений слов на основе больших данных. Эти модели показывают значительное улучшение точности в задачах измерения семантической и синтаксической близости слов по сравнению с предыдущими методами, включая нейронные 

## LLM as a judge

In [38]:
# scores [1-3] чем меньше тем лучше
scores_tbank = [1, 3, 1, 3, 2, 3, 3, 2, 3, 1]
scores_yandex = [3, 1, 2, 2, 3, 2, 2, 1, 2, 3]
scores_qwen = [2, 2, 3, 1, 1, 1, 1, 3, 1, 2]

In [39]:
df_summarization['summarize_tbank_rank'] = scores_tbank
df_summarization['summarize_yandex_rank'] = scores_yandex
df_summarization['summarize_qwen_rank'] = scores_qwen

In [53]:
lst = ['gt',
 'Tbank',
 'YandexGPT',
 'Qwen3',
'summarize_tbank_rank',
 'summarize_yandex_rank',
 'summarize_qwen_rank',
 'sim_scores_tbank',
 'sim_scores_yandex',
 ]

In [56]:
df_summarization = df_summarization[lst]

In [59]:
df_summarization

,gt,Tbank,YandexGPT,Qwen3,summarize_tbank_rank,summarize_yandex_rank,summarize_qwen_rank,sim_scores_tbank,sim_scores_yandex
0,В данной работе представлен метод Test-Time Re...,В статье представлен метод Test-Time Reinforc...,В статье представлен метод Test-Time Reinforce...,В данной работе предложен новый метод обучения...,1,3,2,0.909943,0.930276
1,В статье представлен Transformer — первая архи...,В статье представлен новый архитектурный подх...,В работе представлена новая архитектура нейрон...,Статья представляет архитектуру Transformer — ...,3,1,2,0.797422,0.852751
2,В работе предлагается подход к обучению рекоме...,В данной работе исследуется влияние долгосроч...,В статье изучается влияние долговременной памя...,В статье исследуется влияние долгосрочной памя...,1,2,3,0.774107,0.773998
3,Авторы представляют первый систематический обз...,\n\nИсследование посвящено анализу использова...,В статье представлен системный обзор больших я...,Статья представляет собой первый систематическ...,3,2,1,0.755156,0.731499
4,Авторы представляют общий подход к дообучению ...,"В статье рассматриваются модели, сочетающие п...",В статье рассматривается проблема ограниченног...,Статья представляет обобщённый подход к тонкой...,2,3,1,0.737129,0.688586
5,Авторы представляют два простых и эффективных ...,В статье представлены два новых архитектурных...,В статье предложены две новые архитектуры моде...,В статье предложены два новых архитектурных по...,3,2,1,0.739331,0.708769
6,В работе представлено систематическое исследов...,\n\nВ статье рассматриваются и анализируются ...,В статье представлены усовершенствованные диза...,В статье представлены продвинутые дизайн-систе...,3,2,1,0.856174,0.903531
7,"Авторы представляют LADDER — фреймворк, улучша...","В статье представлен LADDER — метод, улучшающ...",В статье представлен фреймворк LADDER для улуч...,"LADDER — это фреймворк, улучшающий способности...",2,1,3,0.804446,0.914699
8,Scalable-Softmax (SSMax) — это новая замена фу...,В статье представлен Scalable-Softmax (SSMax)...,В статье предлагается Scalable-Softmax (SSMax)...,В статье предложен новый метод Scalable-Softma...,3,2,1,0.864461,0.898752
9,Авторы представляют FlashAttention — IO-ориент...,\nFlashAttention — это новый подход к реализа...,В статье предлагается FlashAttention — алгорит...,"FlashAttention — это точный алгоритм внимания,...",1,3,2,0.899040,0.914540


In [61]:
df_summarization[['summarize_tbank_rank', 'summarize_yandex_rank', 'summarize_qwen_rank']]

,summarize_tbank_rank,summarize_yandex_rank,summarize_qwen_rank
0,1,3,2
1,3,1,2
2,1,2,3
3,3,2,1
4,2,3,1
5,3,2,1
6,3,2,1
7,2,1,3
8,3,2,1
9,1,3,2


In [62]:
df_summarization[['summarize_tbank_rank', 'summarize_yandex_rank', 'summarize_qwen_rank']].mean()

summarize_tbank_rank     2.2
summarize_yandex_rank    2.1
summarize_qwen_rank      1.7
dtype: float64

In [63]:
df_summarization.to_csv('summarize_examples.csv', index=False)

# Bleu/Rouge

In [70]:
df = pd.read_csv('summarize_examples.csv')

## Bleu

In [71]:
hyps = df["Tbank"].tolist()
refs = df["gt"].tolist()
bleu = BLEU(tokenize="intl", smooth_method="exp", effective_order=True)   

df["bleu_tbank"] = [
    bleu.sentence_score(h, [r]).score for h, r in zip(hyps, refs)
]

In [72]:
hyps = df["YandexGPT"].tolist()
refs = df["gt"].tolist()
bleu = BLEU(tokenize="intl", smooth_method="exp", effective_order=True)   

df["bleu_yandex"] = [
    bleu.sentence_score(h, [r]).score for h, r in zip(hyps, refs)
]

In [73]:
hyps = df["Qwen3"].tolist()
refs = df["gt"].tolist()
bleu = BLEU(tokenize="intl", smooth_method="exp", effective_order=True)   

df["bleu_qwen"] = [
    bleu.sentence_score(h, [r]).score for h, r in zip(hyps, refs)
]

In [78]:
df[['bleu_tbank', 'bleu_yandex', 'bleu_qwen']]

,bleu_tbank,bleu_yandex,bleu_qwen
0,14.124650,20.594433,21.681487
1,3.249080,6.779259,1.948774
2,9.707592,8.007145,8.230425
3,5.847037,7.050412,14.343329
4,14.852588,2.320887,5.058517
5,6.148636,5.149729,12.760426
6,12.118868,14.483633,17.328665
7,5.896288,14.176252,10.095051
8,8.201007,9.626733,12.271704
9,10.649770,8.815559,1.047562


In [80]:
df[['bleu_tbank', 'bleu_yandex', 'bleu_qwen']].mean()

bleu_tbank      9.079552
bleu_yandex     9.700404
bleu_qwen      10.476594
dtype: float64

In [79]:
df.to_csv('summarize_examples.csv', index=False)

## Rouge

In [154]:
df = pd.read_csv('summarize_examples.csv')

In [162]:
import re
from nltk.stem.snowball import SnowballStemmer

stem = SnowballStemmer("russian")

class RussianTok(tokenizers.Tokenizer):
    _re = re.compile(r"[а-яё]+", re.I)
    def tokenize(self, txt):
        return [stem.stem(w) for w in self._re.findall(txt.lower())]

ru_tok = RussianTok()

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=False,          
    tokenizer=ru_tok            # свой токенайзер
)

In [163]:
pair_scores = [
    scorer.score(ref, hyp)          
    for hyp, ref in zip(df["Tbank"], df["gt"])
]

df["rouge1_f_tbank"] = [s["rouge1"].fmeasure for s in pair_scores]
df["rouge2_f_tbank"] = [s["rouge2"].fmeasure for s in pair_scores]
df["rougeL_f_tbank"] = [s["rougeL"].fmeasure for s in pair_scores]

df["rouge1_p_tbank"] = [s["rouge1"].precision for s in pair_scores]
df["rouge2_p_tbank"] = [s["rouge2"].precision for s in pair_scores]
df["rougeL_p_tbank"] = [s["rougeL"].precision for s in pair_scores]

df["rouge1_r_tbank"] = [s["rouge1"].recall for s in pair_scores]
df["rouge2_r_tbank"] = [s["rouge2"].recall for s in pair_scores]
df["rougeL_r_tbank"] = [s["rougeL"].recall for s in pair_scores]

In [164]:
pair_scores = [
    scorer.score(ref, hyp)          
    for hyp, ref in zip(df["YandexGPT"], df["gt"])
]

df["rouge1_f_yandex"] = [s["rouge1"].fmeasure for s in pair_scores]
df["rouge2_f_yandex"] = [s["rouge2"].fmeasure for s in pair_scores]
df["rougeL_f_yandex"] = [s["rougeL"].fmeasure for s in pair_scores]

df["rouge1_p_yandex"] = [s["rouge1"].precision for s in pair_scores]
df["rouge2_p_yandex"] = [s["rouge2"].precision for s in pair_scores]
df["rougeL_p_yandex"] = [s["rougeL"].precision for s in pair_scores]

df["rouge1_r_yandex"] = [s["rouge1"].recall for s in pair_scores]
df["rouge2_r_yandex"] = [s["rouge2"].recall for s in pair_scores]
df["rougeL_r_yandex"] = [s["rougeL"].recall for s in pair_scores]

In [166]:
pair_scores = [
    scorer.score(ref, hyp)          
    for hyp, ref in zip(df["Qwen3"], df["gt"])
]

df["rouge1_f_qwen"] = [s["rouge1"].fmeasure for s in pair_scores]
df["rouge2_f_qwen"] = [s["rouge2"].fmeasure for s in pair_scores]
df["rougeL_f_qwen"] = [s["rougeL"].fmeasure for s in pair_scores]

df["rouge1_p_qwen"] = [s["rouge1"].precision for s in pair_scores]
df["rouge2_p_qwen"] = [s["rouge2"].precision for s in pair_scores]
df["rougeL_p_qwen"] = [s["rougeL"].precision for s in pair_scores]

df["rouge1_r_qwen"] = [s["rouge1"].recall for s in pair_scores]
df["rouge2_r_qwen"] = [s["rouge2"].recall for s in pair_scores]
df["rougeL_r_qwen"] = [s["rougeL"].recall for s in pair_scores]

### F-measure

In [167]:
df[['rouge1_f_tbank', 'rouge2_f_tbank', 'rougeL_f_tbank']].mean()

rouge1_f_tbank    0.454545
rouge2_f_tbank    0.128164
rougeL_f_tbank    0.288164
dtype: float64

In [168]:
df[['rouge1_f_yandex', 'rouge2_f_yandex', 'rougeL_f_yandex']].mean()

rouge1_f_yandex    0.458288
rouge2_f_yandex    0.126939
rougeL_f_yandex    0.296055
dtype: float64

In [169]:
df[['rouge1_f_qwen', 'rouge2_f_qwen', 'rougeL_f_qwen']].mean()

rouge1_f_qwen    0.460138
rouge2_f_qwen    0.154361
rougeL_f_qwen    0.314638
dtype: float64

### Precision

In [119]:
df[['rouge1_p_tbank', 'rouge2_p_tbank', 'rougeL_p_tbank']].mean()

rouge1_p_tbank    0.423024
rouge2_p_tbank    0.119411
rougeL_p_tbank    0.265858
dtype: float64

In [123]:
df[['rouge1_p_yandex', 'rouge2_p_yandex', 'rougeL_p_yandex']].mean()

rouge1_p_yandex    0.446262
rouge2_p_yandex    0.123050
rougeL_p_yandex    0.287735
dtype: float64

In [124]:
df[['rouge1_p_qwen', 'rouge2_p_qwen', 'rougeL_p_qwen']].mean()

rouge1_p_qwen    0.462337
rouge2_p_qwen    0.150601
rougeL_p_qwen    0.316523
dtype: float64

### Recall

In [125]:
df[['rouge1_r_tbank', 'rouge2_r_tbank', 'rougeL_r_tbank']].mean()

rouge1_r_tbank    0.504198
rouge2_r_tbank    0.141827
rougeL_r_tbank    0.323061
dtype: float64

In [126]:
df[['rouge1_r_yandex', 'rouge2_r_yandex', 'rougeL_r_yandex']].mean()

rouge1_r_yandex    0.483706
rouge2_r_yandex    0.134893
rougeL_r_yandex    0.313183
dtype: float64

In [127]:
df[['rouge1_r_qwen', 'rouge2_r_qwen', 'rougeL_r_qwen']].mean()

rouge1_r_qwen    0.472849
rouge2_r_qwen    0.161655
rougeL_r_qwen    0.322263
dtype: float64

In [129]:
df.to_csv('summarize_examples.csv', index=False)

# Cosine similarity

In [64]:
urls = ['https://arxiv.org/pdf/2504.16084',
        'https://arxiv.org/abs/1706.03762',
        'https://arxiv.org/pdf/2112.02242',
        'https://arxiv.org/pdf/2405.12819',
        'https://arxiv.org/pdf/2005.11401',
        'https://arxiv.org/pdf/1301.3781',
        'https://arxiv.org/pdf/2501.07391',
        'https://arxiv.org/pdf/2503.00735',
        'https://arxiv.org/pdf/2501.19399',
        'https://arxiv.org/pdf/2205.14135']


In [65]:
from sentence_transformers import SentenceTransformer
import torch

# Загрузим SOTA эмбеддер для русских текстов
embedder = SentenceTransformer("ai-forever/FRIDA")

In [15]:
df_summarization = pd.read_csv('summarize_examples.csv')

In [20]:
df_summarization

,gt,Tbank,YandexGPT,summarize_best_llm
0,В данной работе представлен метод Test-Time Re...,В статье представлен метод Test-Time Reinforc...,В статье представлен метод Test-Time Reinforce...,1
1,В статье представлен Transformer — первая архи...,В статье представлен новый архитектурный подх...,В работе представлена новая архитектура нейрон...,1
2,В работе предлагается подход к обучению рекоме...,В данной работе исследуется влияние долгосроч...,В статье изучается влияние долговременной памя...,0
3,Авторы представляют первый систематический обз...,\n\nИсследование посвящено анализу использова...,В статье представлен системный обзор больших я...,1
4,Авторы представляют общий подход к дообучению ...,"В статье рассматриваются модели, сочетающие п...",В статье рассматривается проблема ограниченног...,0
5,Авторы представляют два простых и эффективных ...,В статье представлены два новых архитектурных...,В статье предложены две новые архитектуры моде...,0
6,В работе представлено систематическое исследов...,\n\nВ статье рассматриваются и анализируются ...,В статье представлены усовершенствованные диза...,0
7,"Авторы представляют LADDER — фреймворк, улучша...","В статье представлен LADDER — метод, улучшающ...",В статье представлен фреймворк LADDER для улуч...,1
8,Scalable-Softmax (SSMax) — это новая замена фу...,В статье представлен Scalable-Softmax (SSMax)...,В статье предлагается Scalable-Softmax (SSMax)...,1
9,Авторы представляют FlashAttention — IO-ориент...,\nFlashAttention — это новый подход к реализа...,В статье предлагается FlashAttention — алгорит...,0


## Tbank

In [21]:
reference_answers = df_summarization['gt'].tolist()
generated_answers = df_summarization['Tbank'].tolist()
articles = urls

In [66]:
from tqdm import tqdm
from typing import List, Dict

def evaluate_similarity_frida(
    questions: List[str], 
    generated_answers: List[str],
    reference_answers: List[str]
) -> List[float]:
    """
    Вычисляет косинусную близость между сгенерированными и эталонными ответами
    с помощью FRIDA SentenceTransformer.

    :param questions: Список вопросов
    :param generated_answers: Список ответов, сгенерированных LLM.
    :param reference_answers: Список эталонных ответов.
        
    :return: Косинусная близость по каждой паре.
    """

    inputs = (
        [f"search_document: {text}" for text in generated_answers] +
        [f"search_document: {text}" for text in reference_answers]
    )

    embeddings = embedder.encode(inputs, convert_to_tensor=True)

    gen_embs = embeddings[:len(generated_answers)]
    ref_embs = embeddings[len(generated_answers):]

    sim_scores = (gen_embs @ ref_embs.T).diagonal().tolist()
    print(len(sim_scores))
    for i, (question, generated, reference) in enumerate(zip(questions, generated_answers, reference_answers)):
        
        print(f"Статья: {question}:")
        #print(f"Ожидалось:\n{reference}")
        #print(f"Предсказано:\n{generated}")
        print(f"cosine_similarity: {sim_scores[i]:.4f}\n")
        print(100 * '*')

    avg_sim = sum(sim_scores) / len(sim_scores)
    print("\nОбщая оценка")
    print(f"Средний similarity: {avg_sim:.4f}")
    
    return sim_scores

In [24]:
scores_tbank = evaluate_similarity_frida(articles, generated_answers, reference_answers)

10
Статья: https://arxiv.org/pdf/2504.16084:
cosine_similarity: 0.9099

****************************************************************************************************
Статья: https://arxiv.org/abs/1706.03762:
cosine_similarity: 0.7974

****************************************************************************************************
Статья: https://arxiv.org/pdf/2112.02242:
cosine_similarity: 0.7741

****************************************************************************************************
Статья: https://arxiv.org/pdf/2405.12819:
cosine_similarity: 0.7552

****************************************************************************************************
Статья: https://arxiv.org/pdf/2005.11401:
cosine_similarity: 0.7371

****************************************************************************************************
Статья: https://arxiv.org/pdf/1301.3781:
cosine_similarity: 0.7393

*******************************************************************************

In [25]:
scores_tbank

[0.9099433422088623,
 0.7974216341972351,
 0.7741065621376038,
 0.7551555037498474,
 0.7371289134025574,
 0.7393308281898499,
 0.8561741709709167,
 0.8044455647468567,
 0.8644611239433289,
 0.8990402817726135]

In [26]:
df_summarization['sim_scores_tbank'] = scores_tbank

## YandexGPT

In [28]:
reference_answers = df_summarization['gt'].tolist()
generated_answers = df_summarization['YandexGPT'].tolist()
articles = urls

In [29]:
scores_yandex = evaluate_similarity_frida(articles, generated_answers, reference_answers)

10
Статья: https://arxiv.org/pdf/2504.16084:
cosine_similarity: 0.9303

****************************************************************************************************
Статья: https://arxiv.org/abs/1706.03762:
cosine_similarity: 0.8528

****************************************************************************************************
Статья: https://arxiv.org/pdf/2112.02242:
cosine_similarity: 0.7740

****************************************************************************************************
Статья: https://arxiv.org/pdf/2405.12819:
cosine_similarity: 0.7315

****************************************************************************************************
Статья: https://arxiv.org/pdf/2005.11401:
cosine_similarity: 0.6886

****************************************************************************************************
Статья: https://arxiv.org/pdf/1301.3781:
cosine_similarity: 0.7088

*******************************************************************************

In [30]:
df_summarization['sim_scores_yandex'] = scores_yandex

In [31]:
df_summarization

,gt,Tbank,YandexGPT,summarize_best_llm,sim_scores_tbank,sim_scores_yandex
0,В данной работе представлен метод Test-Time Re...,В статье представлен метод Test-Time Reinforc...,В статье представлен метод Test-Time Reinforce...,1,0.909943,0.930276
1,В статье представлен Transformer — первая архи...,В статье представлен новый архитектурный подх...,В работе представлена новая архитектура нейрон...,1,0.797422,0.852751
2,В работе предлагается подход к обучению рекоме...,В данной работе исследуется влияние долгосроч...,В статье изучается влияние долговременной памя...,0,0.774107,0.773998
3,Авторы представляют первый систематический обз...,\n\nИсследование посвящено анализу использова...,В статье представлен системный обзор больших я...,1,0.755156,0.731499
4,Авторы представляют общий подход к дообучению ...,"В статье рассматриваются модели, сочетающие п...",В статье рассматривается проблема ограниченног...,0,0.737129,0.688586
5,Авторы представляют два простых и эффективных ...,В статье представлены два новых архитектурных...,В статье предложены две новые архитектуры моде...,0,0.739331,0.708769
6,В работе представлено систематическое исследов...,\n\nВ статье рассматриваются и анализируются ...,В статье представлены усовершенствованные диза...,0,0.856174,0.903531
7,"Авторы представляют LADDER — фреймворк, улучша...","В статье представлен LADDER — метод, улучшающ...",В статье представлен фреймворк LADDER для улуч...,1,0.804446,0.914699
8,Scalable-Softmax (SSMax) — это новая замена фу...,В статье представлен Scalable-Softmax (SSMax)...,В статье предлагается Scalable-Softmax (SSMax)...,1,0.864461,0.898752
9,Авторы представляют FlashAttention — IO-ориент...,\nFlashAttention — это новый подход к реализа...,В статье предлагается FlashAttention — алгорит...,0,0.899040,0.914540


## Qwen3

In [67]:
reference_answers = df_summarization['gt'].tolist()
generated_answers = df_summarization['Qwen3'].tolist()
articles = urls

In [70]:
scores_qwen = evaluate_similarity_frida(articles, generated_answers, reference_answers)

10
Статья: https://arxiv.org/pdf/2504.16084:
cosine_similarity: 0.9530

****************************************************************************************************
Статья: https://arxiv.org/abs/1706.03762:
cosine_similarity: 0.8079

****************************************************************************************************
Статья: https://arxiv.org/pdf/2112.02242:
cosine_similarity: 0.7632

****************************************************************************************************
Статья: https://arxiv.org/pdf/2405.12819:
cosine_similarity: 0.7767

****************************************************************************************************
Статья: https://arxiv.org/pdf/2005.11401:
cosine_similarity: 0.7756

****************************************************************************************************
Статья: https://arxiv.org/pdf/1301.3781:
cosine_similarity: 0.8769

*******************************************************************************

In [71]:
df_summarization['sim_scores_qwen'] = scores_qwen

C:\Users\kiril\AppData\Local\Temp\ipykernel_19996\3967206300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summarization['sim_scores_qwen'] = scores_qwen


In [72]:
df_summarization

,gt,Tbank,YandexGPT,Qwen3,summarize_tbank_rank,summarize_yandex_rank,summarize_qwen_rank,sim_scores_tbank,sim_scores_yandex,sim_scores_qwen
0,В данной работе представлен метод Test-Time Re...,В статье представлен метод Test-Time Reinforc...,В статье представлен метод Test-Time Reinforce...,В данной работе предложен новый метод обучения...,1,3,2,0.909943,0.930276,0.953033
1,В статье представлен Transformer — первая архи...,В статье представлен новый архитектурный подх...,В работе представлена новая архитектура нейрон...,Статья представляет архитектуру Transformer — ...,3,1,2,0.797422,0.852751,0.807929
2,В работе предлагается подход к обучению рекоме...,В данной работе исследуется влияние долгосроч...,В статье изучается влияние долговременной памя...,В статье исследуется влияние долгосрочной памя...,1,2,3,0.774107,0.773998,0.763235
3,Авторы представляют первый систематический обз...,\n\nИсследование посвящено анализу использова...,В статье представлен системный обзор больших я...,Статья представляет собой первый систематическ...,3,2,1,0.755156,0.731499,0.776702
4,Авторы представляют общий подход к дообучению ...,"В статье рассматриваются модели, сочетающие п...",В статье рассматривается проблема ограниченног...,Статья представляет обобщённый подход к тонкой...,2,3,1,0.737129,0.688586,0.775555
5,Авторы представляют два простых и эффективных ...,В статье представлены два новых архитектурных...,В статье предложены две новые архитектуры моде...,В статье предложены два новых архитектурных по...,3,2,1,0.739331,0.708769,0.876854
6,В работе представлено систематическое исследов...,\n\nВ статье рассматриваются и анализируются ...,В статье представлены усовершенствованные диза...,В статье представлены продвинутые дизайн-систе...,3,2,1,0.856174,0.903531,0.781188
7,"Авторы представляют LADDER — фреймворк, улучша...","В статье представлен LADDER — метод, улучшающ...",В статье представлен фреймворк LADDER для улуч...,"LADDER — это фреймворк, улучшающий способности...",2,1,3,0.804446,0.914699,0.919719
8,Scalable-Softmax (SSMax) — это новая замена фу...,В статье представлен Scalable-Softmax (SSMax)...,В статье предлагается Scalable-Softmax (SSMax)...,В статье предложен новый метод Scalable-Softma...,3,2,1,0.864461,0.898752,0.911014
9,Авторы представляют FlashAttention — IO-ориент...,\nFlashAttention — это новый подход к реализа...,В статье предлагается FlashAttention — алгорит...,"FlashAttention — это точный алгоритм внимания,...",1,3,2,0.899040,0.914540,0.846391


In [73]:
df_summarization.to_csv('summarize_examples.csv', index=False)

In [75]:
df_summarization[['sim_scores_tbank', 'sim_scores_yandex', 'sim_scores_qwen']].mean()

sim_scores_tbank     0.813721
sim_scores_yandex    0.831740
sim_scores_qwen      0.841162
dtype: float64

In [76]:
df_summarization.to_csv('summarize_examples.csv', index=False)

# Среднее время суммаризации

In [9]:
print(f'Среднее время суммаризации Tbank: 15 сек')
print(f'Среднее время суммаризации Yandex: 12 сек')
print(f'Среднее время суммаризации Qwen3: 17 сек')

Среднее время суммаризации Tbank: 15 сек
Среднее время суммаризации Yandex: 12 сек
Среднее время суммаризации Qwen3: 17 сек


# Средние ранги на основе similarity

In [269]:
df = pd.read_csv('test_results/summarize_examples.csv')
ranked = df[['sim_scores_tbank', 'sim_scores_yandex', 'sim_scores_qwen']].rank(axis=1, method='min', ascending=False)

In [270]:
ranked.mean().sort_values()

sim_scores_qwen      1.7
sim_scores_yandex    2.0
sim_scores_tbank     2.3
dtype: float64

# Оценка генератора

Оценивать будем аналогичным образом с суммаризацией - намайним вопросы по статьям, эталонные ответы на них и в дальнейшем будем применять LLM as a judge + cos similarity подход

## Генерация эталонных ответов по 50 вопросам

### Статья Attention is all you need

In [142]:
arxiv_url = "https://arxiv.org/abs/1706.03762"
arxiv_id = extract_arxiv_id(arxiv_url)

In [143]:
questions = ['Какой оптимайзер и гиперпараметры использовались в модели?',
            'Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?',
             'Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?',
             'Как устроена структура encoder и decoder в Transformer?',
             'Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?',
             'Какие функции выполняют позиционные кодировки и почему были выбраны синусоиды?',
             'В чем преимущества self-attention по сравнению с рекуррентными и сверточными слоями в плане вычислительной сложности и длины пути зависимостей?',
             'Какие техники регуляризации использовались при обучении Transformer?',
             'Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?',
             'Почему Transformer позволяет более эффективную параллелизацию при обучении?'
            ]

In [144]:
gt = ['''В оригинальной статье использовался Adam с параметрами: β₁=0.9, β₂=0.98, ε=1e−9. Learning rate варьировался по scheduler’у с warm-up:
lr = 𝑑 model − 0.5 ⋅ min(step−0.5,step⋅warmup_steps−1.5)lr=d model−0.5​⋅min(step −0.5,step⋅warmup_steps −1.5)
Также использовался dropout (0.1) и label smoothing (ε=0.1).''',
     'Transformer заменяет RNN или LSTM/GRU, использовавшиеся в encoder/decoder, на self-attention и feed-forward слои. Он также убирает необходимость в рекурсии.',
     '''Это механизм внимания, который вычисляет:
Attention(𝑄,𝐾,𝑉)=softmax(𝑄𝐾𝑇𝑑𝑘)Attention(Q,K,V)=softmax(dk​​QK T)
Деление на 𝑑𝑘 предотвращает слишком большие значения dot product, которые могли бы привести к маленьким градиентам из-за saturation softmax.''',
     'Encoder: N одинаковых блоков (обычно N=6), каждый включает: Multi-Head Self-Attention → Add & Norm → Feed-Forward → Add & Norm. Decoder: тоже N блоков, но с дополнительным masked self-attention и encoder-decoder attention.',
     'Потому что каждая "голова" обучается фокусироваться на разных аспектах входа — синтаксис, семантика, позиционные зависимости и т.д., давая более богатое представление.',
     'Позиционные кодировки добавляют информацию о порядке токенов, который теряется в self-attention. Синусоиды позволяют модели экстраполировать на более длинные последовательности, чем те, что были в обучении.',
     'Self-attention: O(n²) по длине последовательности, но все элементы обрабатываются параллельно. RNN: O(n), но последовательно (нет параллелизма). Conv: O(log n) глубина, но требует много фильтров. Self-attention обеспечивает короткий путь между любыми двумя токенами.',
     'Dropout (в attention, feed-forward, embeddings). Label smoothing. Early stopping, при необходимости',
     '''EN→DE: BLEU 28.4 (vs. предыдущие ~26.4)
EN→FR: BLEU 41.8 (vs. ~40.4) 
Transformer превзошел все предыдущие seq2seq модели, включая LSTM + attention.''',
     '''Потому что self-attention не требует последовательной обработки, как в RNN — все токены обрабатываются одновременно, что идеально для GPU/TPU.''']

In [145]:
df = pd.DataFrame()

df['Questions'] = questions
df['gt'] = gt

In [146]:
df.to_csv('test_results/article_1_questions.csv', index=False)

### Статья TTRL

In [147]:
arxiv_url = "https://arxiv.org/pdf/2504.16084"
arxiv_id = extract_arxiv_id(arxiv_url)

In [148]:
questions = ['В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?',
             'Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?',
             'Какую роль играет метод большинства голосов (majority voting) в архитектуре TTRL?',
             'Какие улучшения производительности были достигнуты с помощью TTRL на бенчмарке AIME 2024?',
             'Как TTRL сравнивается с RL на размеченных тестовых данных по эффективности?',
             'Почему TTRL способен работать даже при неточных оценках меток?',
             'Какие модели и бенчмарки использовались для оценки эффективности TTRL?',
             'Какие факторы могут привести к сбою или неэффективности TTRL?',
             'Как TTRL масштабируется при увеличении размера модели?',
             'С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?'
            ]

In [149]:
gt = ['TTRL — это метод обучения с подкреплением без доступа к истинным меткам на тестовых данных. Он позволяет LLM адаптироваться во время инференса, используя собственные предсказания и механизм majority voting для построения псевдонаград.',
     '''Модель многократно генерирует ответы на один и тот же запрос. Ответ, встречающийся чаще всего (majority vote), принимается за "псевдометку", и каждое предсказание получает награду:
1, если совпадает с псевдометкой
0 — если нет.''',
     'Majority voting используется для оценки метки и вычисления награды: он определяет наиболее вероятный правильный ответ на основе собственных сэмплов модели, заменяя реальные метки.',
     'На AIME 2024 модель Qwen2.5-Math-7B улучшилась с 16.7% до 43.3% pass@1, что составляет +159.3% прироста без использования разметки',
     'TTRL почти достигает производительности RL с доступом к истинным меткам (RL-leakage). Это значит, что псевдонаград, полученных через majority voting, достаточно для сильной адаптации модели',
     '''Потому что:
RL устойчив к шуму в наградах.
Даже при ошибочной метке часть предсказаний всё равно может получить корректный негативный сигнал, усиливая обучение.
Reward accuracy может быть высокой даже при низкой label accuracy''',
     '''Модели: Qwen2.5-Math-1.5B, Qwen2.5-Math-7B, LLaMA-3.1-8B-Instruct
Бенчмарки: AIME 2024, AMC, MATH-500''',
     '''Недостаток предварительных знаний у модели (например, у LLaMA-3.1-8B-Instruct).
Неправильные гиперпараметры (batch size, temperature и т.д.)
Сложные и далекие от обучающего распределения тестовые данные''',
     '''TTRL масштабируется положительно — более крупные модели (например, Qwen2.5-Math-7B) показывают больший прирост производительности, т.к. лучше оценивают метки и генерируют более стабильные предсказания''',
     'Совместим с PPO и GRPO. В экспериментах использовались GRPO (основной) и PPO (альтернативный) — оба показали схожую эффективность, но PPO стабильнее']

In [150]:
df = pd.DataFrame()

df['Questions'] = questions
df['gt'] = gt

In [152]:
df.to_csv('test_results/article_2_questions.csv', index=False)

### Статья Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

In [154]:
arxiv_url = "https://arxiv.org/pdf/2005.11401"
arxiv_id = extract_arxiv_id(arxiv_url)

In [155]:
questions = ['Какая модель используется в качестве генератора в RAG?',
             'Какая модель используется в качестве ретривера в RAG и как она инициализируется?',
             'Чем отличаются архитектуры RAG-Token и RAG-Sequence?',
             'Какой объем документов используется в векторном индексе Wikipedia?',
             'Какие задачи были использованы для оценки RAG-моделей?',
             'Почему авторы считают генерацию предпочтительной по сравнению с извлечением при ответе на вопросы?',
             'Какую производительность показал RAG на задаче Jeopardy Question Generation по сравнению с BART?',
             'Как осуществляется совместное обучение генератора и ретривера в RAG?',
             'Какой эффект даёт "hot-swapping" индекса документов в RAG?',
             'Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?'
            ]

In [160]:
gt = ['BART-large (400M параметров) используется как генератор (pθ). Это seq2seq Transformer, обученный с денойзинг-объективом',
     'DPR (Dense Passage Retriever) — bi-encoder на базе BERT-base. Один BERT кодирует запрос (q(x)), другой — документы (d(z)). Инициализируется моделью, обученной на TriviaQA и Natural Questions',
     '''RAG-Sequence использует один и тот же документ для генерации всей последовательности.
RAG-Token может использовать разные документы для каждого токена — более гибкий, но вычислительно затратный вариант''',
     '21 миллион документов, каждая статья разбита на 100-словные чанки. Используется Wikipedia dump от декабря 2018 года',
     '''Open-domain QA: Natural Questions, TriviaQA, WebQuestions, CuratedTrec
Abstractive QA: MS-MARCO
Question Generation: Jeopardy QGen
Classification: FEVER''',
     '''Потому что:
Можно обобщать и генерировать ответ даже при отсутствии точного совпадения в документах.
Ответ может быть фактуальным и разнообразным, даже если он не извлекается напрямую из текста''',
     '''RAG-Token превзошёл BART по метрике Q-BLEU-1 (22.2 против 19.7). Также в human evaluation RAG оказался более фактуальным (42.7%) и специфичным (37.4%), чем BART''',
     'Модель обучается end-to-end, минимизируя отрицательный log-likelihood. Только query-encoder BERTq и BART обучаются, document-encoder BERTd и индекс остаются фиксированными',
     'Можно обновлять знания модели без переобучения, просто заменив векторный индекс. Например, при смене лидеров стран между 2016 и 2018 годами модель корректно меняла ответы при подмене индекса',
     '''RAG-Token: стандартный beam search с суммой по top-K документов на каждом шаге.
RAG-Sequence: beam search по каждому документу отдельно, с дополнительными forward-pass для маргинализации, либо "Fast Decoding" — без пересчёта гипотез, которых нет в биме''']

In [162]:
df = pd.DataFrame()

df['Questions'] = questions
df['gt'] = gt

In [164]:
df.to_csv('test_results/article_3_questions.csv', index=False)

### Статья Efficient Estimation of Word Representations in Vector Space

In [165]:
arxiv_url = "https://arxiv.org/pdf/1301.3781"
arxiv_id = extract_arxiv_id(arxiv_url)

In [166]:
questions = ['Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?',
             'В чём основное различие между архитектурами CBOW и Skip-gram?',
             'Какие типы отношений включены в Semantic-Syntactic Word Relationship test set?',
             'Какие значения точности показала модель Skip-gram на семантических и синтаксических подзадачах?',
             'Какой подход используется в Skip-gram модели для выбора контекстных слов?',
             'Какова формула вычислительной сложности модели CBOW?',
             'Почему авторы отказались от использования скрытого слоя в новых архитектурах?',
             'Как масштабировалась тренировка моделей в распределённой системе DistBelief?',
             'Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?',
             'Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?'
            ]

In [167]:
gt = ['''CBOW (Continuous Bag-of-Words)
Skip-gram
Обе модели являются лог-линейными и лишены скрытого слоя для повышения скорости обучения''',
     '''CBOW предсказывает текущие слово по контексту (среднее по окрестным словам).
Skip-gram наоборот — предсказывает окрестные слова по текущему слову''',
     '''Семантические (5 типов): столицы, валюты, отношения человек-женщина и т.д.
Синтаксические (9 типов): степени сравнения, времена глаголов, национальности и др''',
     '''Для Skip-gram с размерностью 300 и обучением на 783M слов:
Semantic: 50.0%
Syntactic: 55.9%
Total: 53.3%''',
     'Слово предсказывает контекст в окне размера C. Контекст выбирается случайно в пределах окна (от 1 до C). Более дальние слова встречаются реже (subsampling)',
     '''Q=N⋅D+D⋅log 2​(V)
Где:
N — количество слов в контексте
D — размерность векторов
V — размер словаря''',
     'Потому что основная вычислительная нагрузка приходилась на скрытый слой. Удаление скрытого слоя позволило обучать модели на гораздо больших корпусах с миллиардами слов за разумное время',
     '''Использовалась асинхронная mini-batch SGD + AdaGrad
До 100 копий моделей на множестве CPU
Skip-gram 1000D на 6B слов обучался за ~2.5 дня на 125 CPU-ядер''',
     '''Сначала вручную созданы списки пар слов.
Затем автоматически сформированы тысячи вопросов путём случайной комбинации двух пар из одного типа (например, города и штаты).
Используются только однословные термины''',
     '''Применяется векторная арифметика:
vec("king") − vec("man") + vec("woman") ≈ vec("queen")
Ответ — ближайшее по косинусному расстоянию слово к полученному вектору''']

In [168]:
df = pd.DataFrame()

df['Questions'] = questions
df['gt'] = gt

In [170]:
df.to_csv('test_results/article_4_questions.csv', index=False)

### Статья LADDER: SELF-IMPROVING LLMS THROUGH RECURSIVE PROBLEM DECOMPOSITION

In [172]:
arxiv_url = "https://arxiv.org/pdf/2503.00735"
arxiv_id = extract_arxiv_id(arxiv_url)

In [173]:
questions = ['В чём заключается ключевая идея фреймворка LADDER?',
             'Какой метод используется для верификации правильности решений интегралов в LADDER?',
             'Как осуществляется генерация вариантов задач в LADDER?',
             'Что такое TTRL и чем он отличается от LADDER?',
             'Какие результаты достигнуты моделью Qwen2.5 7B на экзамене MIT Integration Bee после применения LADDER и TTRL?',
             'Какая функция вознаграждения используется в GRPO в рамках обучения в LADDER?',
             'Почему RL без использования вариантов задач показывает плохие результаты на задачах интегрирования?',
             'Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?',
             'Как LADDER масштабирует обучение без увеличения размера модели?',
             'Какие ограничения или сложности были замечены при генерации вариантов задач?'
            ]

In [174]:
gt = ['''Модель самостоятельно улучшает свои способности, генерируя и решая проще сформулированные варианты сложных задач. Это создаёт градиент сложности, по которому она учится решать всё более трудные задачи без внешней разметки или помощи человека''',
     'Применяется численная верификация: сравнение ответа модели с численным значением интеграла на 5 случайных отрезках в диапазоне [-10, 10], с учётом сингулярностей и порогом точности 1e−2',
     '''Варианты создаются по дереву:
Применяются 3–5 заранее определённых математических трансформаций.
Используются batch prompting, temperature cycling и персонажи (перспективы) (например, "думай как Эйлер").
Генерация идёт рекурсивно, до 3-х уровней глубины''',
     '''TTRL (Test-Time RL) — это тестовое применение RL: при инференсе модель генерирует варианты текущей задачи и обучается на них.
Разница:
LADDER — обучение на train-наборе заранее.
TTRL — обучение в момент теста на текущей задаче''',
     '''LADDER: 73% (вышел на уровень квалификации)
LADDER + TTRL: 90%, лучший результат среди моделей, включая OpenAI o1 (80%)''',
     '''GRPO использует два компонента:
Accuracy reward: 1, если интеграл решён правильно (по численной верификации).
Format reward: 1, если ответ оформлен в <ANSWER>...</ANSWER>
Преимущества GRPO: без критика, с групповой нормализацией через advantage''',
     'Без градиента сложности RL быстро проваливается: задачи слишком сложны, и отсутствие положительных примеров приводит к катастрофической деградации — до 0% accuracy за 30 шагов обучения',
     '''Batch prompting (по 10 за раз)
Temperature cycling (0.8–1.4)
Persona prompting (разные стили мышления)
Рекурсивная генерация до глубины 3''',
     'Путём генерации тысяч вариантов и обучения на них. Модель (всего 7B параметров) опережает GPT-4o и другие более крупные модели, не увеличивая архитектуру, а лишь правильно направляя обучение',
     '''Примерно 8% вариантов оказались нерешаемыми
Некоторые “облегчённые” варианты на деле оказались сложнее, из-за случайных усложняющих трансформаций
Верификация требовала большого числа проверок и таймаутов
Была потребность в фильтрации “мусорных” ответов (например, символ интеграла как ответ)''']

In [175]:
df = pd.DataFrame()

df['Questions'] = questions
df['gt'] = gt

In [176]:
df.to_csv('test_results/article_5_questions.csv', index=False)

## Генерация ответов моими LLM

In [3]:
def retrieve_and_rerank(question, vectordb, reranker, arxiv_id, top_k=5, top_n=2):
    """
    Извлекает top_k кандидатов из Chroma и реранжирует их с помощью FlagReranker.
    Возвращает top_n наиболее релевантных фрагментов.
    
    :param question: Вопрос пользователя
    :param vectordb: Объект Chroma
    :param top_k: Сколько фрагментов извлекать из Chroma
    :param top_n: Сколько лучших возвращать после rerank
    :return: Список кортежей (Document, score)
    """
    retriever = vectordb.as_retriever(search_kwargs={"k": top_k, "filter": {"arxiv_id": arxiv_id}})
    candidates = retriever.get_relevant_documents(question)

    pairs = [[question, doc.page_content] for doc in candidates]
    scores = reranker.compute_score(pairs, normalize=True)

    reranked = sorted(zip(candidates, scores), key=lambda x: -x[1])
    
    # print("🔍 Rerank результаты:")
    # for i, (doc, score) in enumerate(reranked):
    #     print(f"#Чанк{i+1} — {doc.page_content}...")

    return reranked[:top_n]

### Tbank

In [7]:
model_name = "t-bank-ai/T-lite-instruct-0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quant_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)
terminators = list({
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>") or tokenizer.eos_token_id,
    })

embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )

reranker = FlagReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)

vectorstore = Chroma(persist_directory="test_chroma_db", embedding_function=embedding_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
C:\Users\kiril\AppData\Local\Temp\ipykernel_21172\295889804.py:12: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
C:\Users\kiril\AppData\Local\Temp\ipykernel_21172\295889804.py:19: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="test_chroma_db", embedding_fu

In [38]:
SYSTEM_MSG = (
    "Ты — помощник по научным статьям. "
    "Твоя задача — дать короткий, точный и однозначный ответ на вопрос, "
    "используя только приведённый контекст. "
    "Нельзя продолжать диалог, задавать встречные вопросы или повторяться. Отвечай строго на русском языке"
)


def build_messages(question: str, context: str) -> List[dict]:
    """Формирует сообщения в нужном для chat_template формате."""
    return [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": f"Вопрос:\n{question}"},
        {"role": "system", "content": f"Контекст:\n{context}"},
    ]

#### Attention is all you need

In [19]:
questions = ['Какой оптимайзер и гиперпараметры использовались в модели?',
            'Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?',
             'Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?',
             'Как устроена структура encoder и decoder в Transformer?',
             'Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?',
             'Какие функции выполняют позиционные кодировки и почему были выбраны синусоиды?',
             'В чем преимущества self-attention по сравнению с рекуррентными и сверточными слоями в плане вычислительной сложности и длины пути зависимостей?',
             'Какие техники регуляризации использовались при обучении Transformer?',
             'Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?',
             'Почему Transformer позволяет более эффективную параллелизацию при обучении?'
            ]

In [20]:
arxiv_url = "https://arxiv.org/abs/1706.03762"
arxiv_id = extract_arxiv_id(arxiv_url)

In [21]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, eos_token_id=terminators)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:05<00:51,  5.74s/it]

Вопрос: Какой оптимайзер и гиперпараметры использовались в модели?

Ответ на вопрос: В данном контексте использовался оптимайзер Adam с параметрами: *β* 1 = 0.9, *β* 2 = 0.98 и *ϵ* = 10^(-9). В процессе обучения применялся динамический подход к изменению скорости обучения.




 20%|████████████████▌                                                                  | 2/10 [00:11<00:44,  5.61s/it]

Вопрос: Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?

Ответ на вопрос: В традиционных sequence-to-sequence моделях, таких как те, что используются для машинного перевода, Transformer заменяет рекуррентные слои (RNN) и слои на основе свёрточных сетей (CNN). Вместо них он применяет механизмы многоголового само-внимания (multi-headed self-attention).




 30%|████████████████████████▉                                                          | 3/10 [00:18<00:45,  6.56s/it]

...

Вопрос: Какие техники регуляризации использовались при обучении Transformer?

Ответ на вопрос: Для обучения Transformer использовались следующие техники регуляризации:

1. **Dropout**: Применяется на всех слоях, кроме последнего, с вероятностью 0.1.
2. **Weight Decay**: Регуляризация L2, которая применяется к весам модели, чтобы предотвратить переобучение.
3. **Label Smoothing**: Введение небольшой ошибки в метки целевых данных, что помогает модели лучше обобщать и уменьшает её уверенность в неправильных ответах.

Эти методы помогают улучшить устойчивость и обобщающую способность модели, что особенно важно для больших моделей, таких как Transformer.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:33<00:11, 11.54s/it]

Вопрос: Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?

Ответ на вопрос: На задачах перевода с английского на французский и немецкий языки, модель Transformer показала значительное улучшение по сравнению с предыдущими лидерами. Она достигла высокого BLEU-оценки (41.0) и при этом потребовала меньших ресурсов для обучения, чем предыдущие лучшие модели. Важным фактором стало использование более высокой вероятности dropout (0.1 вместо 0.3), что способствовало лучшей производительности.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:42<00:00, 10.23s/it]

Вопрос: Почему Transformer позволяет более эффективную параллелизацию при обучении?

Ответ на вопрос: Параллелизация в Transformer возможна из-за отсутствия рекуррентных слоев, которые требуют последовательного выполнения и ограничивают возможности распараллеливания. Вместо этого Transformer использует механизм внимания, который может обрабатывать данные параллельно, так как он не требует последовательного обхода входных данных. Это позволяет эффективнее использовать многоядерные и многопроцессорные системы, что ускоряет процесс обучения и улучшает производительность модели.




In [29]:
times_1 = times

In [30]:
np.mean(times_1)

np.float64(10.112690758705138)

In [22]:
df = pd.read_csv('test_results/article_1_questions.csv')

In [31]:
df['Tbank'] = outputs

In [33]:
df.to_csv('test_results/article_1_questions.csv', index=False)

#### Статья TTRL

In [35]:
questions = ['В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?',
             'Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?',
             'Какую роль играет метод большинства голосов (majority voting) в архитектуре TTRL?',
             'Какие улучшения производительности были достигнуты с помощью TTRL на бенчмарке AIME 2024?',
             'Как TTRL сравнивается с RL на размеченных тестовых данных по эффективности?',
             'Почему TTRL способен работать даже при неточных оценках меток?',
             'Какие модели и бенчмарки использовались для оценки эффективности TTRL?',
             'Какие факторы могут привести к сбою или неэффективности TTRL?',
             'Как TTRL масштабируется при увеличении размера модели?',
             'С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?'
            ]

In [36]:
arxiv_url = "https://arxiv.org/pdf/2504.16084"
arxiv_id = extract_arxiv_id(arxiv_url)

In [39]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, eos_token_id=terminators)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:09<01:29,  9.93s/it]

Вопрос: В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?

Ответ на вопрос: Основная идея метода Test-Time Reinforcement Learning (TTRL) заключается в применении обучения с подкреплением (RL) для улучшения больших языковых моделей (LLMs) на основе неотмеченных данных. В отличие от традиционных подходов, TTRL использует информацию из самого процесса инференса и предобученных моделей для оценки и оптимизации производительности модели в режиме реального времени, что позволяет модели самостоятельно адаптироваться и улучшаться без необходимости вручную заданных меток.




 20%|████████████████▌                                                                  | 2/10 [00:17<01:08,  8.58s/it]

Вопрос: Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?

Ответ на вопрос: Оценка вознаграждения в TTRL (Test-Time Reinforcement Learning) происходит через использование методов, которые не требуют точных меток. В частности, применяется Test-Time Scaling (TTS), где для получения полезных оценок применяется, например, метод большинства голосов. Это позволяет создать функцию вознаграждения, которая помогает обучению модели на основе обратной связи, полученной из неотмеченных данных.




 30%|████████████████████████▉                                                          | 3/10 [00:23<00:51,  7.41s/it]

...

Вопрос: Какие факторы могут привести к сбою или неэффективности TTRL?

Ответ на вопрос: Основные причины неэффективности или сбоя в TTRL (Transfer and Training with Reinforcement Learning) связаны с недостатком предварительных знаний о целевой задаче. Это особенно критично, когда тестовые данные сложнее и включают новые элементы, а модель не имеет подходящих механизмов для адаптации и обучения на основе сложного материала. В случае с Qwen2.5-Math-1.5B и LLaMA-3.1-8B-Instruct, отсутствие значительного улучшения может быть следствием недостаточной подготовленности моделей к таким сложным задачам.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:15<00:08,  8.75s/it]

Вопрос: Как TTRL масштабируется при увеличении размера модели?

Ответ на вопрос: TTRL (Test-Time Reinforcement Learning) демонстрирует улучшение производительности при увеличении размера модели. В данном контексте, большие модели (например, 7B параметров) показывают значительные улучшения в задачах, таких как AIME 2024 и AMC, за счет более точного прогнозирования и лучшего обучения на новых данных. Модели меньшего размера (1.5B) не дают столь заметных результатов, что связано с их ограниченными возможностями.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:21<00:00,  8.11s/it]

Вопрос: С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?

Ответ на вопрос: Техника TTRL (Temporal Training with Reinforcement Learning) совместима с алгоритмами, такими как PPO (Proximal Policy Optimization) и GRPO (Generalized Reinforcement Policy Optimization). В экспериментах, представленных в контексте, использовался PPO, который показал стабильные результаты и сопоставимую эффективность с GRPO.




In [40]:
times_2 = times

In [41]:
np.mean(times_2)

np.float64(7.986197686195373)

In [42]:
df = pd.read_csv('test_results/article_2_questions.csv')

In [43]:
df['Tbank'] = outputs

In [45]:
df.to_csv('test_results/article_2_questions.csv', index=False)

#### Статья Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

In [46]:
questions = ['Какая модель используется в качестве генератора в RAG?',
             'Какая модель используется в качестве ретривера в RAG и как она инициализируется?',
             'Чем отличаются архитектуры RAG-Token и RAG-Sequence?',
             'Какой объем документов используется в векторном индексе Wikipedia?',
             'Какие задачи были использованы для оценки RAG-моделей?',
             'Почему авторы считают генерацию предпочтительной по сравнению с извлечением при ответе на вопросы?',
             'Какую производительность показал RAG на задаче Jeopardy Question Generation по сравнению с BART?',
             'Как осуществляется совместное обучение генератора и ретривера в RAG?',
             'Какой эффект даёт "hot-swapping" индекса документов в RAG?',
             'Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?'
            ]

In [47]:
arxiv_url = "https://arxiv.org/pdf/2005.11401"
arxiv_id = extract_arxiv_id(arxiv_url)

In [48]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, eos_token_id=terminators)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:09<01:28,  9.85s/it]

Вопрос: Какая модель используется в качестве генератора в RAG?

Ответ на вопрос: В RAG (Retrieval-Augmented Generation) используется модель, состоящая из двух компонентов: (i) "retriever" (получатель), который на основе входного запроса *x* выбирает и возвращает наиболее релевантные текстовые фрагменты *z*; и (ii) "generator" (генератор), который, используя эти фрагменты и исходный запрос, генерирует целевой текст *y*. Важно отметить, что оба компонента имеют свои параметры и функции вероятности, которые определяют их работу.




 20%|████████████████▌                                                                  | 2/10 [00:15<00:59,  7.39s/it]

Вопрос: Какая модель используется в качестве ретривера в RAG и как она инициализируется?

Ответ на вопрос: В качестве ретривера в RAG используется модель, инициализированная на основе DPR (Dense Passage Retrieval). Для этого применяется обучение на основе Natural Questions и TriviaQA, что позволяет эффективно извлекать и ранжировать релевантные документы.




 30%|████████████████████████▉                                                          | 3/10 [00:25<01:00,  8.71s/it]

...

Вопрос: Как осуществляется совместное обучение генератора и ретривера в RAG?

Ответ на вопрос: В RAG (Retrieval-Augmented Generation) обучение генератора и ретривера происходит совместно. Генератор (например, модель на основе T5 или BART) и система поиска (ретривер) работают вместе, чтобы улучшить качество генерации текста. Оба компонента проходят обучение на одном и том же наборе данных, что позволяет им адаптироваться и взаимодействовать для создания более точных и релевантных ответов.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:04<00:06,  6.92s/it]

Вопрос: Какой эффект даёт "hot-swapping" индекса документов в RAG?

Ответ на вопрос: "Hot-swapping" индекса в RAG (Retrieval-Augmented Generation) позволяет обновлять и заменять базу данных для поиска без необходимости повторного обучения всей модели. Это обеспечивает гибкость и возможность быстрого обновления информации, что важно для поддержания актуальности и точности ответов на запросы.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:14<00:00,  7.40s/it]

Вопрос: Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?

Ответ на вопрос: Для декодирования в RAG-Sequence и RAG-Token используются различные методы аппроксимации. В RAG-Token применяется стандартный автокодировочный подход, где вероятность перехода *p* ( *y|x* ) аппроксимируется через выборку из верхних *k* вероятностей *p* ( *·|x* ) и последующее использование этих значений в рамках стандартного бим декодера. В RAG-Sequence, вероятно, применяется другой метод, но конкретные детали не указаны в данном контексте.




In [49]:
times_3 = times

In [50]:
np.mean(times_3)

np.float64(7.293178129196167)

In [51]:
df = pd.read_csv('test_results/article_3_questions.csv')

In [52]:
df['Tbank'] = outputs

In [54]:
df.to_csv('test_results/article_3_questions.csv', index=False)

#### Статья Efficient Estimation of Word Representations in Vector Space

In [55]:
questions = ['Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?',
             'В чём основное различие между архитектурами CBOW и Skip-gram?',
             'Какие типы отношений включены в Semantic-Syntactic Word Relationship test set?',
             'Какие значения точности показала модель Skip-gram на семантических и синтаксических подзадачах?',
             'Какой подход используется в Skip-gram модели для выбора контекстных слов?',
             'Какова формула вычислительной сложности модели CBOW?',
             'Почему авторы отказались от использования скрытого слоя в новых архитектурах?',
             'Как масштабировалась тренировка моделей в распределённой системе DistBelief?',
             'Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?',
             'Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?'
            ]

In [56]:
arxiv_url = "https://arxiv.org/pdf/1301.3781"
arxiv_id = extract_arxiv_id(arxiv_url)

In [57]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, eos_token_id=terminators)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:10<01:34, 10.48s/it]

Вопрос: Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?

Ответ на вопрос: Авторы предложили две модели для обучения векторных представлений слов: 
1. **Модель на основе нейронной сети с линейным слоем и нелинейным скрытым слоем** (feedforward neural network with a linear projection layer and a non-linear hidden layer). 
2. **Модель, использующая архитектуру нейронной сети для языковых моделей (NNLM)**, которая также применялась в предыдущих исследованиях, но была усовершенствована и адаптирована для более эффективного обучения векторных представлений.




 20%|████████████████▌                                                                  | 2/10 [00:24<01:38, 12.36s/it]

Вопрос: В чём основное различие между архитектурами CBOW и Skip-gram?

Ответ на вопрос: Основное различие между архитектурами CBOW (Continuous Bag of Words) и Skip-gram заключается в их подходе к обучению. CBOW предсказывает текущее слово, используя контекст (окружение) из нескольких слов, тогда как Skip-gram делает обратное: предсказывает ближайшие слова вокруг заданного слова. Это приводит к разным результатам на задачах, требующих анализа семантики и синтаксиса текста. Skip-gram лучше справляется с выявлением семантических связей, а CBOW сильнее в синтаксических задачах.




 30%|████████████████████████▉                                                          | 3/10 [00:33<01:17, 11.13s/it]

...

Вопрос: Как масштабировалась тренировка моделей в распределённой системе DistBelief?

Ответ на вопрос: Масштабирование тренировки моделей в системе DistBelief происходило за счет распределения вычислительных задач между множеством реплик (копий) модели. В частности, использовалась асинхронная градиентная схема с мини-пакетами и адаптивным методом обучения Adagrad. Каждая реплика работала на множестве CPU-ядер, что позволяло эффективно использовать ресурсы кластера. В среднем, применялось от 50 до 100 реплик, что обеспечивало параллельное обучение на больших объемах данных.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:24<00:09,  9.25s/it]

Вопрос: Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?

Ответ на вопрос: Вопросы для тестирования word vectors были сгенерированы в два этапа. Сначала вручную составлялись списки похожих пар слов. Затем, на основе этих списков, создавались вопросы, соединяя случайные пары слов. Например, для создания вопросов о городах и штатах США, использовались списки городов и их соответствующих штатов. В итоге, тестовый набор включает вопросы, которые проверяют как семантические, так и синтаксические связи слов, и охватывает только однословные термины.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:34<00:00,  9.44s/it]

Вопрос: Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?

Ответ на вопрос: Для решения аналогий, таких как "king - man + woman =?", используется метод, основанный на векторных представлениях слов. В этом методе из векторного представления слова "King" вычитается вектор слова "Man", а затем прибавляется вектор слова "Woman". Полученный вектор сравнивается с векторами других слов, и ближайший по смыслу считается ответом. В данном случае, это может быть слово "Queen". 

Таким образом, алгебраические операции над векторами слов позволяют выявить семантическую связь и найти наиболее подходящее слово для завершения аналогии.




In [59]:
times_4 = times

In [60]:
np.mean(times_4)

np.float64(9.327714204788208)

In [61]:
df = pd.read_csv('test_results/article_4_questions.csv')

In [62]:
df['Tbank'] = outputs

In [64]:
df.to_csv('test_results/article_4_questions.csv', index=False)

#### Статья LADDER: SELF-IMPROVING LLMS THROUGH RECURSIVE PROBLEM DECOMPOSITION

In [65]:
questions = ['В чём заключается ключевая идея фреймворка LADDER?',
             'Какой метод используется для верификации правильности решений интегралов в LADDER?',
             'Как осуществляется генерация вариантов задач в LADDER?',
             'Что такое TTRL и чем он отличается от LADDER?',
             'Какие результаты достигнуты моделью Qwen2.5 7B на экзамене MIT Integration Bee после применения LADDER и TTRL?',
             'Какая функция вознаграждения используется в GRPO в рамках обучения в LADDER?',
             'Почему RL без использования вариантов задач показывает плохие результаты на задачах интегрирования?',
             'Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?',
             'Как LADDER масштабирует обучение без увеличения размера модели?',
             'Какие ограничения или сложности были замечены при генерации вариантов задач?'
            ]

In [66]:
arxiv_url = "https://arxiv.org/pdf/2503.00735"
arxiv_id = extract_arxiv_id(arxiv_url)

In [67]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, eos_token_id=terminators)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:07<01:05,  7.30s/it]

Вопрос: В чём заключается ключевая идея фреймворка LADDER?

Ответ на вопрос: Ключевая идея фреймворка LADDER заключается в использовании рекурсивного подхода к обучению, где модель сама генерирует и решает упрощенные версии сложных задач. Это позволяет создать постепенный градиент сложности, что способствует более эффективному и самостоятельному процессу обучения, не требующему значительных вычислительных ресурсов или внешнего контроля.




 20%|████████████████▌                                                                  | 2/10 [00:11<00:46,  5.76s/it]

Вопрос: Какой метод используется для верификации правильности решений интегралов в LADDER?

Ответ на вопрос: Для верификации решений интегралов в LADDER используется численный метод проверки, который сочетает точность с эффективностью вычислений. Этот процесс обеспечивает надежную проверку на различных типах интегралов, учитывая особенности и возможные численные проблемы.




 30%|████████████████████████▉                                                          | 3/10 [00:21<00:51,  7.38s/it]

...

Вопрос: Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?

Ответ на вопрос: Для увеличения разнообразия генерируемых задач использовались две ключевые техники:

1. **Температурное изменение (Temperature Cycling):** В процессе генерации вариантов применялся динамический диапазон температур от 0.8 до 1.4. Это позволило управлять балансом между новизной и математической корректностью.

2. **Персонифицированные запросы (Persona-based Prompting):** Модель получала инструкции, которые требовали от неё рассматривать задачи с разных точек зрения, например, "подходи как Эйлер" или "анализируй как Гаусс". Это способствовало более широкому спектру решений и подходов.

Таким образом, комбинация этих методов и гибкость в управлении количеством генерируемых вариантов (N) обеспечили эффективное разнообразие и качество задач.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:12<00:09,  9.20s/it]

Вопрос: Как LADDER масштабирует обучение без увеличения размера модели?

Ответ на вопрос: LADDER масштабирует обучение, используя методы рекурсивного разложения задач и обучения на основе подкрепления. Модель разбивает сложные проблемы на более простые части, что позволяет ей адаптироваться и совершенствоваться в процессе выполнения задач, не требуя увеличения размера самой модели или дополнительных данных. Это способствует динамическому развитию навыков и повышению эффективности без необходимости в значительном росте ресурсов.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:20<00:00,  8.02s/it]

Вопрос: Какие ограничения или сложности были замечены при генерации вариантов задач?

Ответ на вопрос: Основные сложности при генерации задач связаны с тем, что небольшие изменения в коэффициентах или структуре функций могут значительно повысить сложность интегралов. Например, изменение коэффициента в рациональной функции может привести к появлению корней, которые усложняют вычисления. Это показывает, что даже незначительные модификации могут сильно влиять на уровень трудности задачи.




In [68]:
times_5 = times

In [69]:
np.mean(times_5)

np.float64(7.925498390197754)

In [70]:
df = pd.read_csv('test_results/article_5_questions.csv')

In [71]:
df['Tbank'] = outputs

In [73]:
df.to_csv('test_results/article_5_questions.csv', index=False)

In [15]:
print(f'Среднее время генерации ответа Tbank: 8.5 сек')

Среднее время генерации ответа Tbank: 8.5 сек


### YandexGPT 5

In [5]:
model_name = "yandex/YandexGPT-5-Lite-8B-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
quant_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config, device_map="auto")

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)

embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )

reranker = FlagReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)

vectorstore = Chroma(persist_directory="test_chroma_db", embedding_function=embedding_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0
C:\Users\kiril\AppData\Local\Temp\ipykernel_8864\1072813799.py:8: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
C:\Users\kiril\AppData\Local\Temp\ipykernel_8864\1072813799.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="test_chroma_db", embedding_fun

In [6]:
SYSTEM_MSG = (
    "Ты — помощник по научным статьям. "
    "Твоя задача — дать короткий, точный и однозначный ответ на вопрос, "
    "используя только приведённый контекст. "
    "Нельзя продолжать диалог, задавать встречные вопросы или повторяться. Отвечай строго на русском языке"
)


def build_messages(question: str, context: str) -> List[dict]:
    """Формирует сообщения в нужном для chat_template формате."""
    return [
        {"role": "user", "content": SYSTEM_MSG},
        {"role": "user", "content": f"Вопрос:\n{question}"},
        {"role": "user", "content": f"Контекст:\n{context}"},
    ]

#### Attention is all you need

In [9]:
questions = ['Какой оптимайзер и гиперпараметры использовались в модели?',
            'Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?',
             'Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?',
             'Как устроена структура encoder и decoder в Transformer?',
             'Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?',
             'Какие функции выполняют позиционные кодировки и почему были выбраны синусоиды?',
             'В чем преимущества self-attention по сравнению с рекуррентными и сверточными слоями в плане вычислительной сложности и длины пути зависимостей?',
             'Какие техники регуляризации использовались при обучении Transformer?',
             'Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?',
             'Почему Transformer позволяет более эффективную параллелизацию при обучении?'
            ]

In [10]:
arxiv_url = "https://arxiv.org/abs/1706.03762"
arxiv_id = extract_arxiv_id(arxiv_url)

In [13]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                             | 0/10 [00:00<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 1/10 [00:03<00:32,  3.66s/it]


Вопрос: Какой оптимайзер и гиперпараметры использовались в модели?

Ответ на вопрос:  Мы использовали оптимизатор Adam с параметрами β1 = 0,9, β2 = 0,98 и ϵ = 10^[-9]^.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 2/10 [00:06<00:24,  3.05s/it]


Вопрос: Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?

Ответ на вопрос:  Transformer заменяет рекуррентные слои, наиболее часто используемые в архитектурах encoder-decoder, на механизм multi-headed self-attention.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 3/10 [00:15<00:40,  5.78s/it]


...

Вопрос: Какие техники регуляризации использовались при обучении Transformer?

Ответ на вопрос:  При обучении Transformer использовались три типа регуляризации.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.████████████▋        | 9/10 [00:43<00:04,  4.52s/it]


Вопрос: Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?

Ответ на вопрос:  На задаче перевода EN→FR по WMT 2014 модель Transformer (big) достигла показателя BLEU 41,0, превзойдя все ранее опубликованные одиночные модели при меньшей в четыре раза стоимости обучения по сравнению с предыдущей передовой моделью.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:47<00:00,  4.74s/it]

Вопрос: Почему Transformer позволяет более эффективную параллелизацию при обучении?

Ответ на вопрос:  Transformer позволяет значительно больше параллелизации, поскольку полностью полагается на механизм внимания для выявления глобальных зависимостей между входными и выходными данными, избегая при этом рекуррентных связей.




In [16]:
times_1 = times

In [17]:
np.mean(times_1)

np.float64(4.636782193183899)

In [18]:
df = pd.read_csv('test_results/article_1_questions.csv')

In [20]:
df['YandexGPT'] = outputs

In [22]:
df.to_csv('test_results/article_1_questions.csv', index=False)

#### Статья TTRL

In [23]:
questions = ['В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?',
             'Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?',
             'Какую роль играет метод большинства голосов (majority voting) в архитектуре TTRL?',
             'Какие улучшения производительности были достигнуты с помощью TTRL на бенчмарке AIME 2024?',
             'Как TTRL сравнивается с RL на размеченных тестовых данных по эффективности?',
             'Почему TTRL способен работать даже при неточных оценках меток?',
             'Какие модели и бенчмарки использовались для оценки эффективности TTRL?',
             'Какие факторы могут привести к сбою или неэффективности TTRL?',
             'Как TTRL масштабируется при увеличении размера модели?',
             'С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?'
            ]

In [24]:
arxiv_url = "https://arxiv.org/pdf/2504.16084"
arxiv_id = extract_arxiv_id(arxiv_url)

In [25]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                             | 0/10 [00:00<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 1/10 [00:06<00:55,  6.20s/it]


Вопрос: В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?

Ответ на вопрос:  Основная идея метода Test-Time Reinforcement Learning (TTRL) заключается в тренировке больших языковых моделей (LLMs) с помощью RL на немаркированных данных. TTRL позволяет моделям «самоэволюционировать», используя предварительные знания, заложенные в предварительно обученных моделях.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 2/10 [00:10<00:39,  4.97s/it]


Вопрос: Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?

Ответ на вопрос:  Оценка вознаграждения в TTRL без доступа к истинным меткам осуществляется с помощью распространённых практик в Test-Time Scaling (TTS), таких как majority voting (мажоритарное голосование).




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 3/10 [00:13<00:30,  4.36s/it]


...

Вопрос: Какие факторы могут привести к сбою или неэффективности TTRL?

Ответ на вопрос:  Фактором, который может привести к сбою или неэффективности TTRL, является недостаток предварительных знаний модели для выполнения целевой задачи из-за сложности данных.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.████████████▋        | 9/10 [00:42<00:04,  4.72s/it]


Вопрос: Как TTRL масштабируется при увеличении размера модели?

Ответ на вопрос:  TTRL естественно масштабируется: с увеличением размера модели (от 1,5B до 7B) наблюдается прирост производительности на AIME 2024 и AMC. Большие модели могут производить более точные вознаграждения за большинство голосов во время самосовершенствования, что приводит к более эффективному обучению на новых данных.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:45<00:00,  4.53s/it]

Вопрос: С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?

Ответ на вопрос:  TTRL совместим с разными алгоритмами обучения с подкреплением. В экспериментах использовался алгоритм PPO (Schulman et al., 2017).




In [26]:
times_2 = times

In [27]:
np.mean(times_2)

np.float64(4.416370105743408)

In [28]:
df = pd.read_csv('test_results/article_2_questions.csv')

In [29]:
df['YandexGPT'] = outputs

In [31]:
df.to_csv('test_results/article_2_questions.csv', index=False)

#### Статья Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

In [32]:
questions = ['Какая модель используется в качестве генератора в RAG?',
             'Какая модель используется в качестве ретривера в RAG и как она инициализируется?',
             'Чем отличаются архитектуры RAG-Token и RAG-Sequence?',
             'Какой объем документов используется в векторном индексе Wikipedia?',
             'Какие задачи были использованы для оценки RAG-моделей?',
             'Почему авторы считают генерацию предпочтительной по сравнению с извлечением при ответе на вопросы?',
             'Какую производительность показал RAG на задаче Jeopardy Question Generation по сравнению с BART?',
             'Как осуществляется совместное обучение генератора и ретривера в RAG?',
             'Какой эффект даёт "hot-swapping" индекса документов в RAG?',
             'Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?'
            ]

In [33]:
arxiv_url = "https://arxiv.org/pdf/2005.11401"
arxiv_id = extract_arxiv_id(arxiv_url)

In [34]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                             | 0/10 [00:00<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 1/10 [00:03<00:29,  3.32s/it]


Вопрос: Какая модель используется в качестве генератора в RAG?

Ответ на вопрос:  В RAG используется генератор $p_{\theta} (y_i|x, z, y_{1:i−1})$.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 2/10 [00:06<00:25,  3.15s/it]


Вопрос: Какая модель используется в качестве ретривера в RAG и как она инициализируется?

Ответ на вопрос:  В RAG в качестве ретривера используется ретривер из DPR, который инициализируется с помощью контроля извлечения на Natural Questions и TriviaQA.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 3/10 [00:13<00:35,  5.13s/it]


...

Вопрос: Как осуществляется совместное обучение генератора и ретривера в RAG?

Ответ на вопрос:  Оба генератора и ретривера в RAG обучаются совместно путём тонкой настройки на любой seq2seq задаче.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.████████████▋        | 9/10 [00:31<00:03,  3.07s/it]


Вопрос: Какой эффект даёт "hot-swapping" индекса документов в RAG?

Ответ на вопрос:  «Hot-swapping» индекса документов в RAG позволяет обновлять модель без необходимости переобучения.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:37<00:00,  3.79s/it]

Вопрос: Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?

Ответ на вопрос:  При декодировании в RAG-Sequence и RAG-Token используются различные способы аппроксимации arg max $y$ $p$ ($y|x$). В частности, для RAG-Token применяется подход с использованием стандартного beam decoder и переходных вероятностей с учётом top-$k$ ($p$ ($·|x$)).




In [35]:
times_3 = times

In [36]:
np.mean(times_3)

np.float64(3.6867867946624755)

In [37]:
df = pd.read_csv('test_results/article_3_questions.csv')

In [38]:
df['YandexGPT'] = outputs

In [39]:
df.to_csv('test_results/article_3_questions.csv', index=False)

#### Статья Efficient Estimation of Word Representations in Vector Space

In [40]:
questions = ['Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?',
             'В чём основное различие между архитектурами CBOW и Skip-gram?',
             'Какие типы отношений включены в Semantic-Syntactic Word Relationship test set?',
             'Какие значения точности показала модель Skip-gram на семантических и синтаксических подзадачах?',
             'Какой подход используется в Skip-gram модели для выбора контекстных слов?',
             'Какова формула вычислительной сложности модели CBOW?',
             'Почему авторы отказались от использования скрытого слоя в новых архитектурах?',
             'Как масштабировалась тренировка моделей в распределённой системе DistBelief?',
             'Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?',
             'Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?'
            ]

In [41]:
arxiv_url = "https://arxiv.org/pdf/1301.3781"
arxiv_id = extract_arxiv_id(arxiv_url)

In [42]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                             | 0/10 [00:00<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 1/10 [00:04<00:42,  4.76s/it]


Вопрос: Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?

Ответ на вопрос:  Авторы предложили две модели архитектур: 
1. Модель на основе полносвязной нейронной сети с линейным проекционным слоем и нелинейным скрытым слоем.
2. Другая модельная архитектура для вычисления непрерывных векторных представлений слов.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 2/10 [00:07<00:26,  3.33s/it]


Вопрос: В чём основное различие между архитектурами CBOW и Skip-gram?

Ответ на вопрос:  CBOW предсказывает текущее слово на основе контекста, а Skip-gram предсказывает окружающие слова по заданному текущему слову.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 3/10 [00:11<00:25,  3.62s/it]


...

Вопрос: Как масштабировалась тренировка моделей в распределённой системе DistBelief?

Ответ на вопрос:  В системе DistBelief тренировка моделей масштабировалась за счёт запуска множества реплик одной и той же модели параллельно. Использовалось от 50 до 100 реплик модели во время тренировки. Каждая реплика синхронизирует свои обновления градиента через централизованный сервер, который хранит все параметры.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.████████████▋        | 9/10 [00:36<00:04,  4.33s/it]


Вопрос: Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?

Ответ на вопрос:  Вопросы в тестовом наборе для оценки word vectors были созданы в два этапа: сначала вручную был составлен список похожих пар слов, затем на их основе был сформирован большой список вопросов путём соединения двух пар слов.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:38<00:00,  3.81s/it]

Вопрос: Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?

Ответ на вопрос:  Метод word offset, где выполняются простые алгебраические операции над векторами слов.




In [43]:
times_4 = times

In [44]:
np.mean(times_4)

np.float64(3.7225289344787598)

In [45]:
df = pd.read_csv('test_results/article_4_questions.csv')

In [46]:
df['YandexGPT'] = outputs

In [47]:
df.to_csv('test_results/article_4_questions.csv', index=False)

#### Статья LADDER: SELF-IMPROVING LLMS THROUGH RECURSIVE PROBLEM DECOMPOSITION

In [48]:
questions = ['В чём заключается ключевая идея фреймворка LADDER?',
             'Какой метод используется для верификации правильности решений интегралов в LADDER?',
             'Как осуществляется генерация вариантов задач в LADDER?',
             'Что такое TTRL и чем он отличается от LADDER?',
             'Какие результаты достигнуты моделью Qwen2.5 7B на экзамене MIT Integration Bee после применения LADDER и TTRL?',
             'Какая функция вознаграждения используется в GRPO в рамках обучения в LADDER?',
             'Почему RL без использования вариантов задач показывает плохие результаты на задачах интегрирования?',
             'Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?',
             'Как LADDER масштабирует обучение без увеличения размера модели?',
             'Какие ограничения или сложности были замечены при генерации вариантов задач?'
            ]

In [49]:
arxiv_url = "https://arxiv.org/pdf/2503.00735"
arxiv_id = extract_arxiv_id(arxiv_url)

In [50]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                             | 0/10 [00:00<?, ?it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 1/10 [00:04<00:37,  4.14s/it]


Вопрос: В чём заключается ключевая идея фреймворка LADDER?

Ответ на вопрос:  Ключевая идея фреймворка LADDER заключается в том, что модели могут самостоятельно улучшать свои способности к решению задач путём рекурсивного генерирования и решения постепенно упрощающихся вариантов сложных задач.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 2/10 [00:05<00:21,  2.75s/it]


Вопрос: Какой метод используется для верификации правильности решений интегралов в LADDER?

Ответ на вопрос:  Для верификации правильности решений интегралов в LADDER используется численный метод интеграции.




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.                     | 3/10 [00:10<00:23,  3.41s/it]


...

Вопрос: Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?

Ответ на вопрос:  Для увеличения разнообразия генерируемых вариантов задач использовались следующие техники:
1. Температурный цикл (Temperature cycling) — динамическое изменение температуры выборки между 0,8 и 1,4 для разных запросов.
2. Персонифицированные подсказки (Persona-based prompting) — модель получала задание принять разные математические перспективы (например, «думай как Эйлер, фокусируясь на рядах», «подходи как Гаусс, ища закономерности»).




Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.████████████▋        | 9/10 [00:42<00:04,  4.91s/it]


Вопрос: Как LADDER масштабирует обучение без увеличения размера модели?

Ответ на вопрос:  LADDER масштабирует обучение без увеличения размера модели за счёт рекурсивного разложения задач на более простые подзадачи и использования обучения с подкреплением с проверяемыми вознаграждениями.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.67s/it]

Вопрос: Какие ограничения или сложности были замечены при генерации вариантов задач?

Ответ на вопрос:  При генерации вариантов задач были замечены сложности с контролем качества. Модель генерировала интегралы, сложность которых существенно отличалась от заданного уровня. Небольшие изменения в коэффициентах или составе функций могли превратить кажущиеся простыми интегралы в гораздо более сложные.




In [51]:
times_5 = times

In [52]:
np.mean(times_5)

np.float64(4.579136824607849)

In [53]:
df = pd.read_csv('test_results/article_5_questions.csv')

In [54]:
df['YandexGPT'] = outputs

In [55]:
df.to_csv('test_results/article_5_questions.csv', index=False)

In [56]:
print(f'Среднее время генерации ответа Yandex: {np.mean(times_1 + times_2 + times_3 + times_4 + times_5):.1f} сек')

Среднее время генерации ответа Yandex: 4.2 сек


### QWEN 3

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
quant_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #quantization_config=quant_config,
    torch_dtype="auto",
    device_map="cuda"
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=False)

embedding_model = HuggingFaceBgeEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )

reranker = FlagReranker("BAAI/bge-reranker-v2-m3", use_fp16=True)

vectorstore = Chroma(persist_directory="test_chroma_db", embedding_function=embedding_model)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda
C:\Users\kiril\AppData\Local\Temp\ipykernel_4284\1997649636.py:16: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
C:\Users\kiril\AppData\Local\Temp\ipykernel_4284\1997649636.py:23: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="test_chroma_db", embedding_func

In [5]:
SYSTEM_MSG = (
    "Ты — помощник по научным статьям. "
    "Твоя задача — дать короткий, точный и однозначный ответ на вопрос, используя только приведённый контекст. "
    "Нельзя продолжать диалог, задавать встречные вопросы или повторяться. Отвечай строго на русском языке"
)


def build_messages(question: str, context: str) -> List[dict]:
    """Формирует сообщения в нужном для chat_template формате."""
    return [
        {"role": "system", "content": SYSTEM_MSG},
        {"role": "user", "content": f"Вопрос:\n{question}"},
        {"role": "assistant", "content": f"Контекст:\n{context}"}
    ]

#### Attention is all you need

In [6]:
questions = ['Какой оптимайзер и гиперпараметры использовались в модели?',
            'Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?',
             'Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?',
             'Как устроена структура encoder и decoder в Transformer?',
             'Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?',
             'Какие функции выполняют позиционные кодировки и почему были выбраны синусоиды?',
             'В чем преимущества self-attention по сравнению с рекуррентными и сверточными слоями в плане вычислительной сложности и длины пути зависимостей?',
             'Какие техники регуляризации использовались при обучении Transformer?',
             'Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?',
             'Почему Transformer позволяет более эффективную параллелизацию при обучении?'
            ]

In [7]:
arxiv_url = "https://arxiv.org/abs/1706.03762"
arxiv_id = extract_arxiv_id(arxiv_url)

In [8]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
    start = time.time()
    
    answer = llm(messages, max_new_tokens=256, do_sample=False, temperature=None, top_p=None, top_k=None)[0]['generated_text']
    
    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

C:\Users\kiril\AppData\Local\Temp\ipykernel_4284\2189633747.py:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  candidates = retriever.get_relevant_documents(question)
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 10%|████████▎                                                                          | 1/10 [00:11<01:40, 11.18s/it]

Вопрос: Какой оптимайзер и гиперпараметры использовались в модели?

Ответ на вопрос: Оптимайзер — Adam, гиперпараметры: β₁ = 0.9, β₂ = 0.98, ε = 10⁻⁹.




 20%|████████████████▌                                                                  | 2/10 [00:16<01:00,  7.52s/it]

Вопрос: Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?

Ответ на вопрос: Рекуррентные слои.




 30%|████████████████████████▉                                                          | 3/10 [00:29<01:10, 10.13s/it]

...

Вопрос: Какие техники регуляризации использовались при обучении Transformer?

Ответ на вопрос: В обучении Transformer использовались три типа регуляризации.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:40<00:10, 10.37s/it]

Вопрос: Какие результаты Transformer показал на задачах перевода EN→DE и EN→FR по сравнению с предыдущими SOTA?

Ответ на вопрос: Transformer показал BLEU-счет 41.8 на задаче EN→FR, что превзошло предыдущие SOTA модели.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:53<00:00, 11.33s/it]

Вопрос: Почему Transformer позволяет более эффективную параллелизацию при обучении?

Ответ на вопрос: Потому что Transformer использует механизм внимания, который позволяет моделировать зависимости между любыми позициями входа и выхода за постоянное число операций, в отличие от моделей с рекуррентными или свёрточными сетями, где число операций растёт с расстоянием между позициями. Это позволяет более эффективно использовать параллельные вычисления.




In [9]:
times_1 = times

In [10]:
np.mean(times_1)

np.float64(8.321885013580323)

In [11]:
df = pd.read_csv('test_results/article_1_questions.csv')

In [13]:
df['Qwen3'] = outputs

In [14]:
df.to_csv('test_results/article_1_questions.csv', index=False)

#### Статья TTRL

In [15]:
questions = ['В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?',
             'Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?',
             'Какую роль играет метод большинства голосов (majority voting) в архитектуре TTRL?',
             'Какие улучшения производительности были достигнуты с помощью TTRL на бенчмарке AIME 2024?',
             'Как TTRL сравнивается с RL на размеченных тестовых данных по эффективности?',
             'Почему TTRL способен работать даже при неточных оценках меток?',
             'Какие модели и бенчмарки использовались для оценки эффективности TTRL?',
             'Какие факторы могут привести к сбою или неэффективности TTRL?',
             'Как TTRL масштабируется при увеличении размера модели?',
             'С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?'
            ]

In [16]:
arxiv_url = "https://arxiv.org/pdf/2504.16084"
arxiv_id = extract_arxiv_id(arxiv_url)

In [18]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None, top_k=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:12<01:51, 12.36s/it]

Вопрос: В чем заключается основная идея метода Test-Time Reinforcement Learning (TTRL)?

Ответ на вопрос: Основная идея метода Test-Time Reinforcement Learning (TTRL) заключается в обучении больших языковых моделей с использованием усилительного обучения (RL) на тестовых данных без доступа к меткам, где награды оцениваются на основе согласованности предсказаний модели (метод большинства).




 20%|████████████████▌                                                                  | 2/10 [00:22<01:27, 10.96s/it]

Вопрос: Каким бразом осуществляется оценка вознаграждения в TTRL без доступа к истинным меткам?

Ответ на вопрос: Оценка вознаграждения в TTRL осуществляется с использованием методов, таких как большинство голосов, для оценки меток и вычисления рулеточных вознаграждений на основе правил, без доступа к истинным меткам.




 30%|████████████████████████▉                                                          | 3/10 [00:31<01:11, 10.27s/it]

...

Вопрос: Какие факторы могут привести к сбою или неэффективности TTRL?

Ответ на вопрос: Недостаток предварительных знаний о целевой задаче, сложность тестовых данных, зависимость от приоров, риски коллапса, ограниченность метода оценки меток через голосование и редкость и неизвестность тестовых данных.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:28<00:10, 10.51s/it]

Вопрос: Как TTRL масштабируется при увеличении размера модели?

Ответ на вопрос: TTRL естественно масштабируется с увеличением размера модели, так как более крупные модели способны генерировать более точные награды при голосовании большинством, что приводит к более эффективному обучению на новых данных.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:35<00:00,  9.51s/it]

Вопрос: С какими алгоритмами обучения с подкреплением совместим TTRL и какие из них использовались в экспериментах?

Ответ на вопрос: TTRL совместим с алгоритмами PPO и GRPO, и в экспериментах использовался PPO.




In [19]:
times_2 = times

In [20]:
np.mean(times_2)

np.float64(6.568811774253845)

In [21]:
df = pd.read_csv('test_results/article_2_questions.csv')

In [22]:
df['Qwen3'] = outputs

In [23]:
df.to_csv('test_results/article_2_questions.csv', index=False)

#### Статья Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

In [24]:
questions = ['Какая модель используется в качестве генератора в RAG?',
             'Какая модель используется в качестве ретривера в RAG и как она инициализируется?',
             'Чем отличаются архитектуры RAG-Token и RAG-Sequence?',
             'Какой объем документов используется в векторном индексе Wikipedia?',
             'Какие задачи были использованы для оценки RAG-моделей?',
             'Почему авторы считают генерацию предпочтительной по сравнению с извлечением при ответе на вопросы?',
             'Какую производительность показал RAG на задаче Jeopardy Question Generation по сравнению с BART?',
             'Как осуществляется совместное обучение генератора и ретривера в RAG?',
             'Какой эффект даёт "hot-swapping" индекса документов в RAG?',
             'Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?'
            ]

In [25]:
arxiv_url = "https://arxiv.org/pdf/2005.11401"
arxiv_id = extract_arxiv_id(arxiv_url)

In [26]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None, top_k=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:05<00:51,  5.73s/it]

Вопрос: Какая модель используется в качестве генератора в RAG?

Ответ на вопрос: RAG-Sequence модель.




 20%|████████████████▌                                                                  | 2/10 [00:15<01:05,  8.24s/it]

Вопрос: Какая модель используется в качестве ретривера в RAG и как она инициализируется?

Ответ на вопрос: В RAG в качестве ретривера используется Dense Passage Retriever (DPR), инициализированный с использованием предобучения на данных Natural Questions и TriviaQA.




 30%|████████████████████████▉                                                          | 3/10 [00:27<01:08,  9.73s/it]

...

Вопрос: Как осуществляется совместное обучение генератора и ретривера в RAG?

Ответ на вопрос: Совместное обучение генератора и ретривера в RAG осуществляется через минимизацию отрицательного маргинального логарифмического правдоподобия целевых выходов с использованием стохастического градиентного спуска, при этом документовый энкодер фиксируется, а только query энкодер и генератор подвергаются тонкой настройке.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:31<00:11, 11.20s/it]

Вопрос: Какой эффект даёт "hot-swapping" индекса документов в RAG?

Ответ на вопрос: "Hot-swapping" индекса документов в RAG позволяет обновлять знания модели в режиме реального времени без необходимости переобучения, что обеспечивает актуальность информации при сохранении структуры и параметров модели.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:44<00:00, 10.40s/it]

Вопрос: Какие аппроксимации используются при декодировании в RAG-Sequence и RAG-Token?

Ответ на вопрос: В RAG-Sequence используется аппроксимация top-K для маргинализации вероятности, а в RAG-Token — аппроксимация с использованием top-K и последовательного декодирования с учетом ретривированных документов.




In [27]:
times_3 = times

In [28]:
np.mean(times_3)

np.float64(7.817001175880432)

In [29]:
df = pd.read_csv('test_results/article_3_questions.csv')

In [30]:
df['Qwen3'] = outputs

In [31]:
df.to_csv('test_results/article_3_questions.csv', index=False)

#### Статья Efficient Estimation of Word Representations in Vector Space

In [32]:
questions = ['Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?',
             'В чём основное различие между архитектурами CBOW и Skip-gram?',
             'Какие типы отношений включены в Semantic-Syntactic Word Relationship test set?',
             'Какие значения точности показала модель Skip-gram на семантических и синтаксических подзадачах?',
             'Какой подход используется в Skip-gram модели для выбора контекстных слов?',
             'Какова формула вычислительной сложности модели CBOW?',
             'Почему авторы отказались от использования скрытого слоя в новых архитектурах?',
             'Как масштабировалась тренировка моделей в распределённой системе DistBelief?',
             'Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?',
             'Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?'
            ]

In [33]:
arxiv_url = "https://arxiv.org/pdf/1301.3781"
arxiv_id = extract_arxiv_id(arxiv_url)

In [34]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None, top_k=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:10<01:30, 10.07s/it]

Вопрос: Какие две модели архитектур были предложены авторами для обучения векторных представлений слов?

Ответ на вопрос: Две модели архитектур, предложенные авторами, — это модель с линейным проекционным слоем и нелинейным скрытым слоем, а также модель, основанная на других типах нейронных сетей.




 20%|████████████████▌                                                                  | 2/10 [00:28<01:57, 14.72s/it]

Вопрос: В чём основное различие между архитектурами CBOW и Skip-gram?

Ответ на вопрос: Основное различие между архитектурами CBOW и Skip-gram заключается в том, что CBOW предсказывает текущее слово на основе контекста, а Skip-gram предсказывает окружающие слова на основе текущего слова.




 30%|████████████████████████▉                                                          | 3/10 [00:38<01:29, 12.84s/it]

...

Вопрос: Как масштабировалась тренировка моделей в распределённой системе DistBelief?

Ответ на вопрос: Модели масштабировались с использованием 50 до 100 реплик модели, каждая из которых использовала несколько CPU-ядер на разных машинах в центре данных.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:34<00:10, 10.01s/it]

Вопрос: Как были сгенерированы вопросы в тестовом наборе для оценки word vectors?

Ответ на вопрос: Вопросы в тестовом наборе были сгенерированы в два этапа: сначала создавалась ручная список пар схожих слов, а затем формировался большой список вопросов путём случайного соединения двух пар слов.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:45<00:00, 10.53s/it]

Вопрос: Какой метод используется для ответа на аналогии вроде “king - man + woman = ?”?

Ответ на вопрос: Метод, используемый для ответа на аналогии вроде “king - man + woman =?”, — это метод векторных операций, основанный на алгебраических действиях с векторами слов.




In [35]:
times_4 = times

In [36]:
np.mean(times_4)

np.float64(7.517814183235169)

In [37]:
df = pd.read_csv('test_results/article_4_questions.csv')

In [38]:
df['Qwen3'] = outputs

In [39]:
df.to_csv('test_results/article_4_questions.csv', index=False)

#### Статья LADDER: SELF-IMPROVING LLMS THROUGH RECURSIVE PROBLEM DECOMPOSITION

In [40]:
questions = ['В чём заключается ключевая идея фреймворка LADDER?',
             'Какой метод используется для верификации правильности решений интегралов в LADDER?',
             'Как осуществляется генерация вариантов задач в LADDER?',
             'Что такое TTRL и чем он отличается от LADDER?',
             'Какие результаты достигнуты моделью Qwen2.5 7B на экзамене MIT Integration Bee после применения LADDER и TTRL?',
             'Какая функция вознаграждения используется в GRPO в рамках обучения в LADDER?',
             'Почему RL без использования вариантов задач показывает плохие результаты на задачах интегрирования?',
             'Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?',
             'Как LADDER масштабирует обучение без увеличения размера модели?',
             'Какие ограничения или сложности были замечены при генерации вариантов задач?'
            ]

In [41]:
arxiv_url = "https://arxiv.org/pdf/2503.00735"
arxiv_id = extract_arxiv_id(arxiv_url)

In [42]:
outputs = []
times = []

for question in tqdm(questions):
    reranked = retrieve_and_rerank(question, vectorstore, reranker, arxiv_id, top_k=10, top_n=2)
    top_chunks = [doc.page_content for doc, score in reranked]
    context = "\n\n".join(top_chunks)

    messages = build_messages(question, context)
    messages = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True, enable_thinking=False)
    start = time.time()

    answer = llm(messages, max_new_tokens=256, do_sample=False, top_p=None, temperature=None, top_k=None)[0]['generated_text']

    end = time.time()
    inf_time = end - start
    
    times.append(inf_time)
    print(f'Вопрос: {question}\n')
    print(f'Ответ на вопрос: {answer}\n\n')
    outputs.append(answer)

 10%|████████▎                                                                          | 1/10 [00:09<01:25,  9.55s/it]

Вопрос: В чём заключается ключевая идея фреймворка LADDER?

Ответ на вопрос: Ключевая идея фреймворка LADDER заключается в автономном улучшении способности моделей решать задачи через рекурсивное разложение сложных задач на более простые варианты и обучение на их решениях.




 20%|████████████████▌                                                                  | 2/10 [00:16<01:05,  8.18s/it]

Вопрос: Какой метод используется для верификации правильности решений интегралов в LADDER?

Ответ на вопрос: Метод численной интеграции используется для верификации правильности решений интегралов в LADDER.




 30%|████████████████████████▉                                                          | 3/10 [00:26<01:01,  8.78s/it]

...

Вопрос: Какие техники использовались для увеличения разнообразия генерируемых вариантов задач?

Ответ на вопрос: Использовались следующие техники: случайное выборка преобразований, циклическое изменение температуры при генерации и персонализированные промпты с разными математическими перспективами.




 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:32<00:10, 10.25s/it]

Вопрос: Как LADDER масштабирует обучение без увеличения размера модели?

Ответ на вопрос: LADDER масштабирует обучение, позволяя моделям динамически разбивать сложные задачи на более простые и обучаться на них во время инференса, что позволяет значительно улучшать производительность без увеличения размера модели или предобучения.




100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:42<00:00, 10.27s/it]

Вопрос: Какие ограничения или сложности были замечены при генерации вариантов задач?

Ответ на вопрос: При генерации вариантов задач наблюдались сложности с контролем уровня сложности, так как небольшие изменения в коэффициентах или функциональной составляющей могли значительно увеличивать сложность интегралов.




In [43]:
times_5 = times

In [44]:
np.mean(times_5)

np.float64(8.391678547859192)

In [45]:
df = pd.read_csv('test_results/article_5_questions.csv')

In [46]:
df['Qwen3'] = outputs

In [47]:
df.to_csv('test_results/article_5_questions.csv', index=False)

In [48]:
print(f'Среднее время генерации ответа Qwen3: {np.mean(times_1 + times_2 + times_3 + times_4 + times_5):.1f} сек')

Среднее время генерации ответа Qwen3: 7.7 сек


# Среднее время генерации ответов

In [49]:
print(f'Среднее время генерации ответов Tbank: 8.5 сек')
print(f'Среднее время генерации ответов Yandex: 4.2 сек')
print(f'Среднее время генерации ответов Qwen3: 7.7 сек')

Среднее время генерации ответов Tbank: 8.5 сек
Среднее время генерации ответов Yandex: 4.2 сек
Среднее время генерации ответов Qwen3: 7.7 сек


## Собираем все ответы в один датафрейм

In [52]:
df1 = pd.read_csv('test_results/article_1_questions.csv')
df2 = pd.read_csv('test_results/article_2_questions.csv')
df3 = pd.read_csv('test_results/article_3_questions.csv')
df4 = pd.read_csv('test_results/article_4_questions.csv')
df5 = pd.read_csv('test_results/article_5_questions.csv')

In [53]:
df = pd.concat([df1, df2, df3, df4, df5])

In [57]:
df.to_csv('test_results/article_questions_all.csv', index=False)

## LLM as a judge

In [1]:
import pandas as pd

df = pd.read_csv('article_questions_all.csv')

In [2]:
df

,Questions,gt,Tbank,YandexGPT,Qwen3,gen_tbank_rank,gen_yandex_rank,gen_qwen_rank,sim_scores_tbank,sim_scores_yandex,sim_scores_qwen
0,Какой оптимайзер и гиперпараметры использовали...,В оригинальной статье использовался Adam с пар...,В данном контексте использовался оптимайзер Ad...,Мы использовали оптимизатор Adam с параметрам...,"Оптимайзер — Adam, гиперпараметры: β₁ = 0.9, β...",1,2,2,0.669123,0.570089,0.500410
1,Какие архитектурные компоненты заменяет Transf...,"Transformer заменяет RNN или LSTM/GRU, использ...","В традиционных sequence-to-sequence моделях, т...","Transformer заменяет рекуррентные слои, наибо...",Рекуррентные слои.,1,2,3,0.695416,0.769922,0.370163
2,Что такое Scaled Dot-Product Attention и зачем...,"Это механизм внимания, который вычисляет:\nAtt...",Деление на √dk в Scaled Dot-Product Attention ...,Scaled Dot-Product Attention — это метод вним...,Scaled Dot-Product Attention — это механизм вн...,3,2,1,0.503097,0.558233,0.644848
3,Как устроена структура encoder и decoder в Tra...,"Encoder: N одинаковых блоков (обычно N=6), каж...",Структура Transformer состоит из двух основных...,Структура encoder и decoder в Transformer сос...,Encoder состоит из стека из 6 идентичных слоев...,3,2,1,0.355634,0.415566,0.558227
4,Почему Multi-Head Attention улучшает производи...,"Потому что каждая ""голова"" обучается фокусиров...",Multi-Head Attention улучшает производительнос...,Multi-head attention позволяет модели совмест...,Multi-Head Attention улучшает производительнос...,2,3,1,0.405849,0.377725,0.400534
5,Какие функции выполняют позиционные кодировки ...,Позиционные кодировки добавляют информацию о п...,"Позиционные кодировки (PE) в нейронных сетях, ...",Позиционные кодировки выполняют функцию предс...,Позиционные кодировки позволяют модели учитыва...,3,2,1,0.602244,0.709361,0.736253
6,В чем преимущества self-attention по сравнению...,Self-attention: O(n²) по длине последовательно...,Преимущества self-attention перед рекуррентным...,В плане вычислительной сложности слой self-at...,Self-attention имеет постоянное число операций...,2,3,1,0.619954,0.577273,0.644929
7,Какие техники регуляризации использовались при...,"Dropout (в attention, feed-forward, embeddings...",Для обучения Transformer использовались следую...,При обучении Transformer использовались три т...,В обучении Transformer использовались три типа...,1,2,2,0.342940,0.224844,0.229979
8,Какие результаты Transformer показал на задача...,EN→DE: BLEU 28.4 (vs. предыдущие ~26.4)\nEN→FR...,На задачах перевода с английского на французск...,На задаче перевода EN→FR по WMT 2014 модель T...,Transformer показал BLEU-счет 41.8 на задаче E...,3,2,1,0.672282,0.662603,0.662860
9,Почему Transformer позволяет более эффективную...,Потому что self-attention не требует последова...,Параллелизация в Transformer возможна из-за от...,Transformer позволяет значительно больше пара...,Потому что Transformer использует механизм вни...,1,3,2,0.395121,0.322822,0.407397


In [3]:
df

,Questions,gt,Tbank,YandexGPT,Qwen3,gen_tbank_rank,gen_yandex_rank,gen_qwen_rank,sim_scores_tbank,sim_scores_yandex,sim_scores_qwen
0,Какой оптимайзер и гиперпараметры использовали...,В оригинальной статье использовался Adam с пар...,В данном контексте использовался оптимайзер Ad...,Мы использовали оптимизатор Adam с параметрам...,"Оптимайзер — Adam, гиперпараметры: β₁ = 0.9, β...",1,2,2,0.669123,0.570089,0.500410
1,Какие архитектурные компоненты заменяет Transf...,"Transformer заменяет RNN или LSTM/GRU, использ...","В традиционных sequence-to-sequence моделях, т...","Transformer заменяет рекуррентные слои, наибо...",Рекуррентные слои.,1,2,3,0.695416,0.769922,0.370163
2,Что такое Scaled Dot-Product Attention и зачем...,"Это механизм внимания, который вычисляет:\nAtt...",Деление на √dk в Scaled Dot-Product Attention ...,Scaled Dot-Product Attention — это метод вним...,Scaled Dot-Product Attention — это механизм вн...,3,2,1,0.503097,0.558233,0.644848
3,Как устроена структура encoder и decoder в Tra...,"Encoder: N одинаковых блоков (обычно N=6), каж...",Структура Transformer состоит из двух основных...,Структура encoder и decoder в Transformer сос...,Encoder состоит из стека из 6 идентичных слоев...,3,2,1,0.355634,0.415566,0.558227
4,Почему Multi-Head Attention улучшает производи...,"Потому что каждая ""голова"" обучается фокусиров...",Multi-Head Attention улучшает производительнос...,Multi-head attention позволяет модели совмест...,Multi-Head Attention улучшает производительнос...,2,3,1,0.405849,0.377725,0.400534
5,Какие функции выполняют позиционные кодировки ...,Позиционные кодировки добавляют информацию о п...,"Позиционные кодировки (PE) в нейронных сетях, ...",Позиционные кодировки выполняют функцию предс...,Позиционные кодировки позволяют модели учитыва...,3,2,1,0.602244,0.709361,0.736253
6,В чем преимущества self-attention по сравнению...,Self-attention: O(n²) по длине последовательно...,Преимущества self-attention перед рекуррентным...,В плане вычислительной сложности слой self-at...,Self-attention имеет постоянное число операций...,2,3,1,0.619954,0.577273,0.644929
7,Какие техники регуляризации использовались при...,"Dropout (в attention, feed-forward, embeddings...",Для обучения Transformer использовались следую...,При обучении Transformer использовались три т...,В обучении Transformer использовались три типа...,1,2,2,0.342940,0.224844,0.229979
8,Какие результаты Transformer показал на задача...,EN→DE: BLEU 28.4 (vs. предыдущие ~26.4)\nEN→FR...,На задачах перевода с английского на французск...,На задаче перевода EN→FR по WMT 2014 модель T...,Transformer показал BLEU-счет 41.8 на задаче E...,3,2,1,0.672282,0.662603,0.662860
9,Почему Transformer позволяет более эффективную...,Потому что self-attention не требует последова...,Параллелизация в Transformer возможна из-за от...,Transformer позволяет значительно больше пара...,Потому что Transformer использует механизм вни...,1,3,2,0.395121,0.322822,0.407397


In [93]:
n = 27
str_question = df['Questions'].iloc[n]
str_gt = df['gt'].iloc[n]
str_tbank = df['Tbank'].iloc[n]
str_yandex = df['YandexGPT'].iloc[n]
str_qwen = df['Qwen3'].iloc[n]

print(f'Вопрос: {str_question}\n')
print(f'Эталонный ответ: {str_gt}\n')
print(f'Ответ Tbank: {str_tbank}\n')
print(f'Ответ YandexGPT: {str_yandex}\n')  
print(f'Ответ Qwen: {str_qwen}')

Вопрос: Как осуществляется совместное обучение генератора и ретривера в RAG?

Эталонный ответ: Модель обучается end-to-end, минимизируя отрицательный log-likelihood. Только query-encoder BERTq и BART обучаются, document-encoder BERTd и индекс остаются фиксированными

Ответ Tbank: В RAG (Retrieval-Augmented Generation) обучение генератора и ретривера происходит совместно. Генератор (например, модель на основе T5 или BART) и система поиска (ретривер) работают вместе, чтобы улучшить качество генерации текста. Оба компонента проходят обучение на одном и том же наборе данных, что позволяет им адаптироваться и взаимодействовать для создания более точных и релевантных ответов.

Ответ YandexGPT:  Оба генератора и ретривера в RAG обучаются совместно путём тонкой настройки на любой seq2seq задаче.

Ответ Qwen: Совместное обучение генератора и ретривера в RAG осуществляется через минимизацию отрицательного маргинального логарифмического правдоподобия целевых выходов с использованием стохастическо

In [192]:
# scores [1-3] чем меньше тем лучше
scores_tbank = [1, 1, 3, 3, 2, 3, 2, 1, 3, 1, 2, 2, 1, 3, 2, 3, 1, 2, 3, 1, 3, 3, 1, 1, 2, 3, 2, 3, 2, 3, 3, 2, 2, 3, 3, 1, 3, 1, 1, 1, 3, 1, 1, 3, 1, 3, 3, 1, 3, 3]
scores_yandex = [2, 2, 2, 2, 3, 2, 3, 2, 2, 3, 3, 2, 3, 2, 1, 2, 2, 3, 2, 3, 3, 2, 2, 3, 3, 1, 1, 2, 3, 2, 3, 1, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, 1, 2, 1, 2, 2, 2, 1]
scores_qwen = [2, 3, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 3, 1, 3, 1, 1, 2, 3, 1, 3, 2, 1, 2, 3, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 3, 3, 2, 1, 2, 3, 2, 2, 2, 1, 3, 1, 2]

In [195]:
df['gen_tbank_rank'] = scores_tbank
df['gen_yandex_rank'] = scores_yandex
df['gen_qwen_rank'] = scores_qwen

In [197]:
df.to_csv('test_results/article_questions_all.csv', index=False)

## Средний ранг генерации вопросов

In [201]:
df[['gen_tbank_rank', 'gen_yandex_rank', 'gen_qwen_rank']].mean().sort_values()

gen_qwen_rank      1.74
gen_tbank_rank     2.10
gen_yandex_rank    2.16
dtype: float64

# Bleu/Rouge

In [130]:
df = pd.read_csv('article_questions_all.csv')

## Bleu

In [132]:
hyps = df["Tbank"].tolist()
refs = df["gt"].tolist()
bleu = BLEU(tokenize="intl", smooth_method="exp", effective_order=True)   

df["bleu_tbank"] = [
    bleu.sentence_score(h, [r]).score for h, r in zip(hyps, refs)
]

In [133]:
hyps = df["YandexGPT"].tolist()
refs = df["gt"].tolist()
bleu = BLEU(tokenize="intl", smooth_method="exp", effective_order=True)   

df["bleu_yandex"] = [
    bleu.sentence_score(h, [r]).score for h, r in zip(hyps, refs)
]

In [134]:
hyps = df["Qwen3"].tolist()
refs = df["gt"].tolist()
bleu = BLEU(tokenize="intl", smooth_method="exp", effective_order=True)   

df["bleu_qwen"] = [
    bleu.sentence_score(h, [r]).score for h, r in zip(hyps, refs)
]

In [135]:
df[['bleu_tbank', 'bleu_yandex', 'bleu_qwen']]

,bleu_tbank,bleu_yandex,bleu_qwen
0,3.515791,0.419478,1.273276
1,4.078440,9.514054,0.004866
2,1.934502,1.599418,4.461248
3,2.641894,5.860751,2.088522
4,0.968374,1.622934,1.433300
5,0.973466,2.734225,16.723342
6,3.769086,5.954583,8.624407
7,0.597629,1.232214,1.232214
8,1.053028,4.858588,3.759195
9,2.058758,1.888475,1.959711


In [136]:
df[['bleu_tbank', 'bleu_yandex', 'bleu_qwen']].mean()

bleu_tbank     3.644703
bleu_yandex    4.321198
bleu_qwen      4.840649
dtype: float64

In [137]:
df.to_csv('article_questions_all.csv', index=False)

## Rouge

In [138]:
df = pd.read_csv('article_questions_all.csv')

In [139]:
import re
from nltk.stem.snowball import SnowballStemmer

stem = SnowballStemmer("russian")

class RussianTok(tokenizers.Tokenizer):
    _re = re.compile(r"[а-яё]+", re.I)
    def tokenize(self, txt):
        return [stem.stem(w) for w in self._re.findall(txt.lower())]

ru_tok = RussianTok()

scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"],
    use_stemmer=False,          
    tokenizer=ru_tok            # свой токенайзер
)

In [140]:
pair_scores = [
    scorer.score(ref, hyp)          
    for hyp, ref in zip(df["Tbank"], df["gt"])
]

df["rouge1_f_tbank"] = [s["rouge1"].fmeasure for s in pair_scores]
df["rouge2_f_tbank"] = [s["rouge2"].fmeasure for s in pair_scores]
df["rougeL_f_tbank"] = [s["rougeL"].fmeasure for s in pair_scores]

df["rouge1_p_tbank"] = [s["rouge1"].precision for s in pair_scores]
df["rouge2_p_tbank"] = [s["rouge2"].precision for s in pair_scores]
df["rougeL_p_tbank"] = [s["rougeL"].precision for s in pair_scores]

df["rouge1_r_tbank"] = [s["rouge1"].recall for s in pair_scores]
df["rouge2_r_tbank"] = [s["rouge2"].recall for s in pair_scores]
df["rougeL_r_tbank"] = [s["rougeL"].recall for s in pair_scores]

In [141]:
pair_scores = [
    scorer.score(ref, hyp)          
    for hyp, ref in zip(df["YandexGPT"], df["gt"])
]

df["rouge1_f_yandex"] = [s["rouge1"].fmeasure for s in pair_scores]
df["rouge2_f_yandex"] = [s["rouge2"].fmeasure for s in pair_scores]
df["rougeL_f_yandex"] = [s["rougeL"].fmeasure for s in pair_scores]

df["rouge1_p_yandex"] = [s["rouge1"].precision for s in pair_scores]
df["rouge2_p_yandex"] = [s["rouge2"].precision for s in pair_scores]
df["rougeL_p_yandex"] = [s["rougeL"].precision for s in pair_scores]

df["rouge1_r_yandex"] = [s["rouge1"].recall for s in pair_scores]
df["rouge2_r_yandex"] = [s["rouge2"].recall for s in pair_scores]
df["rougeL_r_yandex"] = [s["rougeL"].recall for s in pair_scores]

In [142]:
pair_scores = [
    scorer.score(ref, hyp)          
    for hyp, ref in zip(df["Qwen3"], df["gt"])
]

df["rouge1_f_qwen"] = [s["rouge1"].fmeasure for s in pair_scores]
df["rouge2_f_qwen"] = [s["rouge2"].fmeasure for s in pair_scores]
df["rougeL_f_qwen"] = [s["rougeL"].fmeasure for s in pair_scores]

df["rouge1_p_qwen"] = [s["rouge1"].precision for s in pair_scores]
df["rouge2_p_qwen"] = [s["rouge2"].precision for s in pair_scores]
df["rougeL_p_qwen"] = [s["rougeL"].precision for s in pair_scores]

df["rouge1_r_qwen"] = [s["rouge1"].recall for s in pair_scores]
df["rouge2_r_qwen"] = [s["rouge2"].recall for s in pair_scores]
df["rougeL_r_qwen"] = [s["rougeL"].recall for s in pair_scores]

### F-measure

In [143]:
df[['rouge1_f_tbank', 'rouge2_f_tbank', 'rougeL_f_tbank']].mean()

rouge1_f_tbank    0.206473
rouge2_f_tbank    0.038891
rougeL_f_tbank    0.154290
dtype: float64

In [144]:
df[['rouge1_f_yandex', 'rouge2_f_yandex', 'rougeL_f_yandex']].mean()

rouge1_f_yandex    0.223548
rouge2_f_yandex    0.037516
rougeL_f_yandex    0.184365
dtype: float64

In [145]:
df[['rouge1_f_qwen', 'rouge2_f_qwen', 'rougeL_f_qwen']].mean()

rouge1_f_qwen    0.255723
rouge2_f_qwen    0.072566
rougeL_f_qwen    0.202847
dtype: float64

### Precision

In [146]:
df[['rouge1_p_tbank', 'rouge2_p_tbank', 'rougeL_p_tbank']].mean()

rouge1_p_tbank    0.151476
rouge2_p_tbank    0.028680
rougeL_p_tbank    0.112875
dtype: float64

In [147]:
df[['rouge1_p_yandex', 'rouge2_p_yandex', 'rougeL_p_yandex']].mean()

rouge1_p_yandex    0.234096
rouge2_p_yandex    0.039478
rougeL_p_yandex    0.195566
dtype: float64

In [148]:
df[['rouge1_p_qwen', 'rouge2_p_qwen', 'rougeL_p_qwen']].mean()

rouge1_p_qwen    0.267341
rouge2_p_qwen    0.073784
rougeL_p_qwen    0.211680
dtype: float64

### Recall

In [149]:
df[['rouge1_r_tbank', 'rouge2_r_tbank', 'rougeL_r_tbank']].mean()

rouge1_r_tbank    0.390185
rouge2_r_tbank    0.071249
rougeL_r_tbank    0.298839
dtype: float64

In [150]:
df[['rouge1_r_yandex', 'rouge2_r_yandex', 'rougeL_r_yandex']].mean()

rouge1_r_yandex    0.259862
rouge2_r_yandex    0.039937
rougeL_r_yandex    0.214634
dtype: float64

In [151]:
df[['rouge1_r_qwen', 'rouge2_r_qwen', 'rougeL_r_qwen']].mean()

rouge1_r_qwen    0.276396
rouge2_r_qwen    0.076814
rougeL_r_qwen    0.221734
dtype: float64

In [152]:
df.to_csv('article_questions_all.csv', index=False)

## Cosine Similarity 

In [208]:
from sentence_transformers import SentenceTransformer
import torch

# Загрузим SOTA эмбеддер для русских текстов
embedder = SentenceTransformer("ai-forever/FRIDA")

In [209]:
df_gen = pd.read_csv('test_results/article_questions_all.csv')

In [216]:
df_gen.head()

,Questions,gt,Tbank,YandexGPT,Qwen3,gen_tbank_rank,gen_yandex_rank,gen_qwen_rank
0,Какой оптимайзер и гиперпараметры использовали...,В оригинальной статье использовался Adam с пар...,В данном контексте использовался оптимайзер Ad...,Мы использовали оптимизатор Adam с параметрам...,"Оптимайзер — Adam, гиперпараметры: β₁ = 0.9, β...",1,2,2
1,Какие архитектурные компоненты заменяет Transf...,"Transformer заменяет RNN или LSTM/GRU, использ...","В традиционных sequence-to-sequence моделях, т...","Transformer заменяет рекуррентные слои, наибо...",Рекуррентные слои.,1,2,3
2,Что такое Scaled Dot-Product Attention и зачем...,"Это механизм внимания, который вычисляет:\nAtt...",Деление на √dk в Scaled Dot-Product Attention ...,Scaled Dot-Product Attention — это метод вним...,Scaled Dot-Product Attention — это механизм вн...,3,2,1
3,Как устроена структура encoder и decoder в Tra...,"Encoder: N одинаковых блоков (обычно N=6), каж...",Структура Transformer состоит из двух основных...,Структура encoder и decoder в Transformer сос...,Encoder состоит из стека из 6 идентичных слоев...,3,2,1
4,Почему Multi-Head Attention улучшает производи...,"Потому что каждая ""голова"" обучается фокусиров...",Multi-Head Attention улучшает производительнос...,Multi-head attention позволяет модели совмест...,Multi-Head Attention улучшает производительнос...,2,3,1


## Tbank

In [217]:
reference_answers = df_gen['gt'].tolist()
generated_answers = df_gen['Tbank'].tolist()
questions = df['Questions'].tolist()

In [220]:
from tqdm import tqdm
from typing import List, Dict

def evaluate_similarity_frida(
    questions: List[str], 
    generated_answers: List[str],
    reference_answers: List[str]
) -> List[float]:
    """
    Вычисляет косинусную близость между сгенерированными и эталонными ответами
    с помощью FRIDA SentenceTransformer.

    :param questions: Список вопросов
    :param generated_answers: Список ответов, сгенерированных LLM.
    :param reference_answers: Список эталонных ответов.
        
    :return: Косинусная близость по каждой паре.
    """

    inputs = (
        [f"search_document: {text}" for text in generated_answers] +
        [f"search_document: {text}" for text in reference_answers]
    )

    embeddings = embedder.encode(inputs, convert_to_tensor=True)

    gen_embs = embeddings[:len(generated_answers)]
    ref_embs = embeddings[len(generated_answers):]

    sim_scores = (gen_embs @ ref_embs.T).diagonal().tolist()
    print(len(sim_scores))
    for i, (question, generated, reference) in enumerate(zip(questions, generated_answers, reference_answers)):
        
        print(f"Вопрос: {question}:")
        print(f"cosine_similarity: {sim_scores[i]:.4f}\n")
        print(100 * '*')

    avg_sim = sum(sim_scores) / len(sim_scores)
    print("\nОбщая оценка")
    print(f"Средний similarity: {avg_sim:.4f}")
    
    return sim_scores

In [221]:
scores_tbank = evaluate_similarity_frida(questions, generated_answers, reference_answers)

50
Вопрос: Какой оптимайзер и гиперпараметры использовались в модели?:
cosine_similarity: 0.6691

****************************************************************************************************
Вопрос: Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?:
cosine_similarity: 0.6954

****************************************************************************************************
Вопрос: Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?:
cosine_similarity: 0.5031

****************************************************************************************************
Вопрос: Как устроена структура encoder и decoder в Transformer?:
cosine_similarity: 0.3556

****************************************************************************************************
Вопрос: Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?:
cosine_similarity: 0.4058

************************

In [223]:
df_gen['sim_scores_tbank'] = scores_tbank

In [225]:
df_gen.head()

,Questions,gt,Tbank,YandexGPT,Qwen3,gen_tbank_rank,gen_yandex_rank,gen_qwen_rank,sim_scores_tbank
0,Какой оптимайзер и гиперпараметры использовали...,В оригинальной статье использовался Adam с пар...,В данном контексте использовался оптимайзер Ad...,Мы использовали оптимизатор Adam с параметрам...,"Оптимайзер — Adam, гиперпараметры: β₁ = 0.9, β...",1,2,2,0.669123
1,Какие архитектурные компоненты заменяет Transf...,"Transformer заменяет RNN или LSTM/GRU, использ...","В традиционных sequence-to-sequence моделях, т...","Transformer заменяет рекуррентные слои, наибо...",Рекуррентные слои.,1,2,3,0.695416
2,Что такое Scaled Dot-Product Attention и зачем...,"Это механизм внимания, который вычисляет:\nAtt...",Деление на √dk в Scaled Dot-Product Attention ...,Scaled Dot-Product Attention — это метод вним...,Scaled Dot-Product Attention — это механизм вн...,3,2,1,0.503097
3,Как устроена структура encoder и decoder в Tra...,"Encoder: N одинаковых блоков (обычно N=6), каж...",Структура Transformer состоит из двух основных...,Структура encoder и decoder в Transformer сос...,Encoder состоит из стека из 6 идентичных слоев...,3,2,1,0.355634
4,Почему Multi-Head Attention улучшает производи...,"Потому что каждая ""голова"" обучается фокусиров...",Multi-Head Attention улучшает производительнос...,Multi-head attention позволяет модели совмест...,Multi-Head Attention улучшает производительнос...,2,3,1,0.405849


## Yandex

In [226]:
reference_answers = df_gen['gt'].tolist()
generated_answers = df_gen['YandexGPT'].tolist()
questions = df['Questions'].tolist()

In [227]:
scores_yandex = evaluate_similarity_frida(questions, generated_answers, reference_answers)

50
Вопрос: Какой оптимайзер и гиперпараметры использовались в модели?:
cosine_similarity: 0.5701

****************************************************************************************************
Вопрос: Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?:
cosine_similarity: 0.7699

****************************************************************************************************
Вопрос: Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?:
cosine_similarity: 0.5582

****************************************************************************************************
Вопрос: Как устроена структура encoder и decoder в Transformer?:
cosine_similarity: 0.4156

****************************************************************************************************
Вопрос: Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?:
cosine_similarity: 0.3777

************************

In [228]:
df_gen['sim_scores_yandex'] = scores_yandex

## Qwen3

In [231]:
reference_answers = df_gen['gt'].tolist()
generated_answers = df_gen['Qwen3'].tolist()
questions = df['Questions'].tolist()

In [233]:
scores_qwen = evaluate_similarity_frida(questions, generated_answers, reference_answers)

50
Вопрос: Какой оптимайзер и гиперпараметры использовались в модели?:
cosine_similarity: 0.5004

****************************************************************************************************
Вопрос: Какие архитектурные компоненты заменяет Transformer в традиционных sequence-to-sequence моделях?:
cosine_similarity: 0.3702

****************************************************************************************************
Вопрос: Что такое Scaled Dot-Product Attention и зачем в нем используется деление на √dk?:
cosine_similarity: 0.6448

****************************************************************************************************
Вопрос: Как устроена структура encoder и decoder в Transformer?:
cosine_similarity: 0.5582

****************************************************************************************************
Вопрос: Почему Multi-Head Attention улучшает производительность по сравнению с одной головой внимания?:
cosine_similarity: 0.4005

************************

In [234]:
df_gen['sim_scores_qwen'] = scores_qwen

In [246]:
df_gen.to_csv('test_results/article_questions_all.csv', index=False)

# Средние sim_scores

In [245]:
df_gen[['sim_scores_tbank', 'sim_scores_yandex', 'sim_scores_qwen']].mean().sort_values(ascending=False)

sim_scores_qwen      0.533568
sim_scores_tbank     0.531741
sim_scores_yandex    0.526195
dtype: float64

# Средние ранги на основе similarity

In [239]:
ranked = df_gen[['sim_scores_tbank', 'sim_scores_yandex', 'sim_scores_qwen']].rank(axis=1, method='min', ascending=False)

In [242]:
ranked.mean().sort_values()

sim_scores_qwen      1.82
sim_scores_tbank     1.98
sim_scores_yandex    2.20
dtype: float64

# Средние ранги на основе собственной проверки

In [241]:
df_gen[['gen_tbank_rank', 'gen_yandex_rank', 'gen_qwen_rank']].mean().sort_values()

gen_qwen_rank      1.74
gen_tbank_rank     2.10
gen_yandex_rank    2.16
dtype: float64

# Соберем все результаты в одну таблицу

In [283]:
df = pd.DataFrame()

In [284]:
df.index = ['YandexGPT 5', 'T-Lite', 'Qwen 3']

In [285]:
df['Средний ранг на основе оценки Gpt4o (суммаризация) (10 статей)'] = [2.1, 2.2, 1.7]
df['Средний ранг на основе косинусной близости (суммаризация) (10 статей)'] = [2.0, 2.3, 1.7]

df['Средний ранг на основе оценки Gpt4o (генерация ответов) (50 вопросов)'] = [2.16, 2.1, 1.74]
df['Средний ранг на основе косинусной близости (генерация ответов) (50 вопросов)'] = [2.2, 1.98, 1.82]

df['Среднее время инференса (суммаризация) (10 статей)'] = [12, 15, 17]
df['Среднее время инференса (генерация ответов) (50 вопросов)'] = [4.2, 8.5, 7.7]

In [286]:
df

,Средний ранг на основе оценки Gpt4o (суммаризация) (10 статей),Средний ранг на основе косинусной близости (суммаризация) (10 статей),Средний ранг на основе оценки Gpt4o (генерация ответов) (50 вопросов),Средний ранг на основе косинусной близости (генерация ответов) (50 вопросов),Среднее время инференса (суммаризация) (10 статей),Среднее время инференса (генерация ответов) (50 вопросов)
YandexGPT 5,2.1,2.0,2.16,2.20,12,4.2
T-Lite,2.2,2.3,2.10,1.98,15,8.5
Qwen 3,1.7,1.7,1.74,1.82,17,7.7


In [287]:
df.to_csv('test_results/agg_results.csv', index=False)